# Hypothesis 3: Hallucination Reduction and Time-to-Market Impact
**Implementing hallucination-reducing techniques in LLMs significantly improves (>30%) time to market in new product development.**

In [ ]:
# Cell [0] - Enhanced Setup and Imports with Advanced Statistical Testing (FIXED)
# Purpose: Import all required libraries and configure environment settings for SiFP COSMIC Estimation Analysis with advanced statistical methods
# Dependencies: pandas, numpy, matplotlib, seaborn, scipy, neo4j, scikit-learn, dotenv, pymc, arviz, bayesian-testing
# Breadcrumbs: Setup -> Environment Configuration -> Analysis Preparation -> Advanced Statistical Methods

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from dotenv import load_dotenv

# Core statistical and machine learning imports
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Database connections
from neo4j import GraphDatabase

# Check scipy version for permutation_test availability
import scipy
scipy_version = [int(x) for x in scipy.__version__.split('.')]
SCIPY_HAS_PERMUTATION_TEST = (scipy_version[0] > 1) or (scipy_version[0] == 1 and scipy_version[1] >= 7)

print(f"SciPy version: {scipy.__version__}")
print(f"Permutation test available: {SCIPY_HAS_PERMUTATION_TEST}")

# Advanced statistical testing imports with better error handling
try:
    # Basic scipy stats (should always be available)
    from scipy.stats import (
        mannwhitneyu, wilcoxon, kruskal, friedmanchisquare, 
        chi2_contingency, fisher_exact, pearsonr, spearmanr, 
        kendalltau, norm, t as t_dist
    )
    
    # Try to import permutation_test and bootstrap (scipy 1.7.0+)
    if SCIPY_HAS_PERMUTATION_TEST:
        from scipy.stats import permutation_test, bootstrap
        PERMUTATION_TEST_AVAILABLE = True
        BOOTSTRAP_AVAILABLE = True
        print("✓ SciPy permutation_test and bootstrap imported successfully")
    else:
        PERMUTATION_TEST_AVAILABLE = False
        BOOTSTRAP_AVAILABLE = False
        print("⚠ SciPy permutation_test not available (requires SciPy >= 1.7.0)")
    
    # Bayesian analysis imports
    try:
        import pymc as pm
        import arviz as az
        BAYESIAN_AVAILABLE = True
        print("✓ PyMC and ArviZ imported successfully")
    except ImportError:
        BAYESIAN_AVAILABLE = False
        print("⚠ PyMC/ArviZ not available")
    
    # Bayesian hypothesis testing
    try:
        from bayesian_testing.experiments import BinaryDataTest, NormalDataTest
        BAYESIAN_TESTING_AVAILABLE = True
        print("✓ Bayesian-testing imported successfully")
    except ImportError:
        BAYESIAN_TESTING_AVAILABLE = False
        print("⚠ bayesian-testing not available")
    
    # Additional statistical utilities (CORRECTED FUNCTION NAMES)
    try:
        from statsmodels.stats.contingency_tables import mcnemar
        from statsmodels.stats.power import ttest_power, tt_ind_solve_power
        from statsmodels.stats.proportion import proportion_confint, proportions_ztest
        from statsmodels.stats.descriptivestats import describe
        from statsmodels.stats.multitest import multipletests
        STATSMODELS_AVAILABLE = True
        print("✓ Statsmodels imported successfully")
    except ImportError:
        STATSMODELS_AVAILABLE = False
        print("⚠ Statsmodels not available")
    
    ADVANCED_STATS_AVAILABLE = True
    print("✓ Basic advanced statistical libraries loaded successfully")
    
except ImportError as e:
    print(f"⚠ Warning: Some statistical libraries not available: {e}")
    ADVANCED_STATS_AVAILABLE = False
    PERMUTATION_TEST_AVAILABLE = False
    BOOTSTRAP_AVAILABLE = False
    BAYESIAN_AVAILABLE = False
    BAYESIAN_TESTING_AVAILABLE = False
    STATSMODELS_AVAILABLE = False

def custom_permutation_test(sample1, sample2, statistic_func, n_resamples=10000, alternative='two-sided', random_state=None):
    """
    Custom implementation of permutation test for older SciPy versions
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    # Calculate observed statistic
    observed_stat = statistic_func(sample1, sample2)
    
    # Combine samples for permutation
    combined = np.concatenate([sample1, sample2])
    n1 = len(sample1)
    
    # Generate permutation distribution
    perm_stats = []
    for _ in range(n_resamples):
        perm_combined = np.random.permutation(combined)
        perm_sample1 = perm_combined[:n1]
        perm_sample2 = perm_combined[n1:]
        perm_stat = statistic_func(perm_sample1, perm_sample2)
        perm_stats.append(perm_stat)
    
    perm_stats = np.array(perm_stats)
    
    # Calculate p-value based on alternative hypothesis
    if alternative == 'two-sided':
        p_value = np.mean(np.abs(perm_stats) >= np.abs(observed_stat))
    elif alternative == 'greater':
        p_value = np.mean(perm_stats >= observed_stat)
    elif alternative == 'less':
        p_value = np.mean(perm_stats <= observed_stat)
    else:
        raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")
    
    # Return result object similar to scipy's permutation_test
    class PermutationTestResult:
        def __init__(self, statistic, pvalue, null_distribution):
            self.statistic = statistic
            self.pvalue = pvalue
            self.null_distribution = null_distribution
    
    return PermutationTestResult(observed_stat, p_value, perm_stats)

def custom_bootstrap_ci(data, statistic_func, n_resamples=10000, confidence_level=0.95, random_state=None):
    """
    Custom implementation of bootstrap confidence intervals
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    # Generate bootstrap samples
    bootstrap_stats = []
    for _ in range(n_resamples):
        bootstrap_sample = np.random.choice(data, len(data), replace=True)
        bootstrap_stat = statistic_func(bootstrap_sample)
        bootstrap_stats.append(bootstrap_stat)
    
    bootstrap_stats = np.array(bootstrap_stats)
    
    # Calculate confidence interval
    alpha = 1 - confidence_level
    lower_percentile = (alpha / 2) * 100
    upper_percentile = (1 - alpha / 2) * 100
    
    ci_lower = np.percentile(bootstrap_stats, lower_percentile)
    ci_upper = np.percentile(bootstrap_stats, upper_percentile)
    
    # Return result object similar to scipy's bootstrap
    class BootstrapResult:
        def __init__(self, confidence_interval, bootstrap_distribution):
            self.confidence_interval = (ci_lower, ci_upper)
            self.bootstrap_distribution = bootstrap_distribution
    
    return BootstrapResult((ci_lower, ci_upper), bootstrap_stats)

def setup_analysis_environment():
    """
    Configure analysis environment with display options and styling
    
    Returns:
        dict: Configuration parameters for the analysis
    """
    # Suppress warnings for cleaner output
    warnings.filterwarnings('ignore')
    
    # Configure matplotlib and seaborn styling
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Configure pandas display options for better readability
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    # Load environment variables
    load_dotenv()
    
    # Configuration parameters
    config = {
        'NEO4J_URI': os.getenv('NEO4J_URI'),
        'NEO4J_USER': os.getenv('NEO4J_USER'),
        'NEO4J_PASSWORD': os.getenv('NEO4J_PASSWORD'),
        'NEO4J_PROJECT_NAME': os.getenv('NEO4J_PROJECT_NAME'),
        'CONVERSION_FACTOR': 0.957,  # SiFP = 0.957 × UFP (Desharnais)
        'COST_PER_HOUR': 100,  # Industry standard for cost impact calculations
        
        # Statistical testing configuration
        'ALPHA_LEVEL': 0.05,  # Significance level
        'BOOTSTRAP_SAMPLES': 10000,  # Number of bootstrap samples
        'PERMUTATION_SAMPLES': 10000,  # Number of permutation samples
        'BAYESIAN_SAMPLES': 2000,  # Number of MCMC samples
        'BAYESIAN_CHAINS': 4,  # Number of MCMC chains
        'IMPROVEMENT_THRESHOLD': 0.30,  # 30% improvement threshold
        'POWER_TARGET': 0.80,  # Target statistical power
        
        # Advanced testing flags
        'ADVANCED_STATS_AVAILABLE': ADVANCED_STATS_AVAILABLE,
        'SCIPY_VERSION': scipy.__version__
    }
    
    print("✓ Analysis environment configured successfully")
    print(f"✓ Project: {config['NEO4J_PROJECT_NAME']}")
    print(f"✓ Advanced statistical methods: {'Available' if ADVANCED_STATS_AVAILABLE else 'Limited'}")
    
    return config

def initialize_statistical_methods():
    """
    Initialize and test statistical methods availability with fallbacks
    
    Returns:
        dict: Available statistical methods configuration
    """
    methods_config = {
        'permutation_tests': False,
        'bootstrap_tests': False,
        'bayesian_analysis': False,
        'power_analysis': False
    }
    
    # Test permutation functionality
    try:
        test_data1 = np.random.normal(0, 1, 10)
        test_data2 = np.random.normal(0, 1, 10)
        
        def test_statistic(x, y):
            return np.mean(x) - np.mean(y)
        
        if PERMUTATION_TEST_AVAILABLE:
            # Use scipy's permutation_test
            perm_result = permutation_test((test_data1, test_data2), test_statistic, 
                                         n_resamples=100, random_state=42)
            methods_config['permutation_tests'] = True
            print("✓ SciPy permutation_test available and working")
        else:
            # Use custom implementation
            perm_result = custom_permutation_test(test_data1, test_data2, test_statistic, 
                                                n_resamples=100, random_state=42)
            methods_config['permutation_tests'] = True
            print("✓ Custom permutation_test fallback available and working")
            
    except Exception as e:
        print(f"⚠ Warning: Permutation test functionality issue: {e}")
        methods_config['permutation_tests'] = False
    
    # Test bootstrap functionality
    try:
        if BOOTSTRAP_AVAILABLE:
            # Use scipy's bootstrap
            bootstrap_result = bootstrap((test_data1,), np.mean, n_resamples=100, 
                                       random_state=42)
            methods_config['bootstrap_tests'] = True
            print("✓ SciPy bootstrap available and working")
        else:
            # Use custom implementation
            bootstrap_result = custom_bootstrap_ci(test_data1, np.mean, n_resamples=100, 
                                                 random_state=42)
            methods_config['bootstrap_tests'] = True
            print("✓ Custom bootstrap fallback available and working")
            
    except Exception as e:
        print(f"⚠ Warning: Bootstrap functionality issue: {e}")
        methods_config['bootstrap_tests'] = False
    
    # Test Bayesian functionality
    if BAYESIAN_AVAILABLE:
        try:
            with pm.Model() as test_model:
                mu = pm.Normal('mu', mu=0, sigma=1)
            methods_config['bayesian_analysis'] = True
            print("✓ Bayesian analysis available and working")
        except Exception as e:
            print(f"⚠ Warning: Bayesian analysis issue: {e}")
            methods_config['bayesian_analysis'] = False
    
    # Test power analysis (CORRECTED FUNCTION CALL)
    if STATSMODELS_AVAILABLE:
        try:
            power_result = tt_ind_solve_power(effect_size=0.5, nobs1=20, alpha=0.05)
            methods_config['power_analysis'] = True
            print("✓ Power analysis available and working")
        except Exception as e:
            print(f"⚠ Warning: Power analysis issue: {e}")
            methods_config['power_analysis'] = False
    
    return methods_config

def setup_bayesian_environment():
    """
    Configure PyMC and ArviZ for Bayesian analysis
    
    Returns:
        dict: Bayesian analysis configuration
    """
    if not BAYESIAN_AVAILABLE:
        return {}
    
    try:
        # Configure ArviZ styling
        az.style.use('arviz-darkgrid')
        
        # Set up PyMC configuration (version-aware)
        try:
            # For PyMC3 compatibility
            pm.set_tt_config('floatX', 'float64')
        except AttributeError:
            # PyMC v4+ doesn't have set_tt_config - configuration is handled differently
            # This is normal and expected for newer PyMC versions
            pass
        
        bayesian_config = {
            'target_accept': 0.9,
            'chains': 4,
            'draws': 2000,
            'tune': 1000,
            'cores': min(4, os.cpu_count() or 1),
            'return_inferencedata': True,
            'random_seed': 42
        }
        
        print("✓ Bayesian analysis environment configured")
        return bayesian_config
        
    except Exception as e:
        print(f"⚠ Warning: Bayesian environment setup issue: {e}")
        return {}

# Execute setup when cell runs
CONFIG = setup_analysis_environment()
STATISTICAL_CONFIG = initialize_statistical_methods()
BAYESIAN_CONFIG = setup_bayesian_environment()

# Make custom functions available globally if needed
if not PERMUTATION_TEST_AVAILABLE:
    permutation_test = custom_permutation_test
    print("✓ Custom permutation_test function registered globally")

if not BOOTSTRAP_AVAILABLE:
    bootstrap = custom_bootstrap_ci
    print("✓ Custom bootstrap function registered globally")

# Display configuration summary
print(f"\n" + "="*60)
print("STATISTICAL ANALYSIS CONFIGURATION")
print("="*60)
print(f"SciPy Version: {scipy.__version__}")
print(f"Available Methods:")
print(f"  ✓ Basic statistics and visualization")
print(f"  ✓ Standard hypothesis testing (t-tests, Mann-Whitney)")
print(f"  {'✓' if STATISTICAL_CONFIG.get('permutation_tests') else '✗'} Permutation tests {'(custom fallback)' if not PERMUTATION_TEST_AVAILABLE and STATISTICAL_CONFIG.get('permutation_tests') else '(scipy native)' if STATISTICAL_CONFIG.get('permutation_tests') else ''}")
print(f"  {'✓' if STATISTICAL_CONFIG.get('bootstrap_tests') else '✗'} Advanced bootstrapping {'(custom fallback)' if not BOOTSTRAP_AVAILABLE and STATISTICAL_CONFIG.get('bootstrap_tests') else '(scipy native)' if STATISTICAL_CONFIG.get('bootstrap_tests') else ''}")
print(f"  {'✓' if STATISTICAL_CONFIG.get('bayesian_analysis') else '✗'} Bayesian hypothesis testing")
print(f"  {'✓' if STATISTICAL_CONFIG.get('power_analysis') else '✗'} Power analysis")

print(f"\nConfiguration Parameters:")
print(f"  Alpha level: {CONFIG['ALPHA_LEVEL']}")
print(f"  Improvement threshold: {CONFIG['IMPROVEMENT_THRESHOLD']:.0%}")
print(f"  Bootstrap samples: {CONFIG['BOOTSTRAP_SAMPLES']:,}")
print(f"  Permutation samples: {CONFIG['PERMUTATION_SAMPLES']:,}")
if BAYESIAN_CONFIG:
    print(f"  MCMC samples: {CONFIG['BAYESIAN_SAMPLES']:,}")
    print(f"  MCMC chains: {CONFIG['BAYESIAN_CHAINS']}")

if not PERMUTATION_TEST_AVAILABLE:
    print(f"\n⚠ NOTE: Using custom permutation test implementation")
    print(f"   To use native SciPy implementation: pip install 'scipy>=1.7.0'")

if not BOOTSTRAP_AVAILABLE:
    print(f"\n⚠ NOTE: Using custom bootstrap implementation")
    print(f"   To use native SciPy implementation: pip install 'scipy>=1.7.0'")

print(f"\n✓ Enhanced statistical analysis environment ready")

In [ ]:
# Cell [1] - Load and Process Java Code Metrics
# Purpose: Load actual implementation metrics from iTrust java.csv for baseline establishment
# Dependencies: pandas, configured environment (Cell 0)
# Breadcrumbs: Setup -> Code Metrics Loading -> Baseline Data Preparation

def load_code_metrics():
    """
    Load and process Java code metrics from iTrust dataset
    
    Returns:
        pd.DataFrame: Processed code metrics with derived calculations
    """
    try:
        # Load java.csv from iTrust dataset
        java_df = pd.read_csv('../datasets/iTrust/iTrust/java.csv')
        print(f"✓ Loaded java.csv: {java_df.shape[0]} code entities")

        # Filter to only File entries (excluding methods, functions, etc.)
        file_df = java_df[java_df['Kind'] == 'File'].copy()
        print(f"  Files: {len(file_df)}")

        # Select relevant metrics for SiFP analysis
        metrics_columns = [
            'Name', 'CountLine', 'CountLineCode', 'CountLineComment',
            'CountDeclClass', 'CountDeclMethod', 'CountDeclMethodAll',
            'CountDeclExecutableUnit', 'Cyclomatic', 'MaxCyclomatic'
        ]

        # Create analysis-ready dataframe
        code_metrics_df = file_df[metrics_columns].copy()

        # Calculate derived metrics that correlate with function points
        code_metrics_df['TotalUnits'] = (
            code_metrics_df['CountDeclClass'] + 
            code_metrics_df['CountDeclMethod']
        )

        return code_metrics_df
        
    except Exception as e:
        print(f"Error loading code metrics: {e}")
        raise

# Load and process the code metrics
code_metrics_df = load_code_metrics()

# Display sample data for verification
print("\nSample code metrics:")
print(code_metrics_df.head())

# Calculate and display summary statistics
print("\nCode Metrics Summary:")
print("=" * 40)
print(f"  Total files: {len(code_metrics_df)}")
print(f"  Average lines of code: {code_metrics_df['CountLineCode'].mean():.0f}")
print(f"  Average methods per file: {code_metrics_df['CountDeclMethod'].mean():.1f}")
print(f"  Average cyclomatic complexity: {code_metrics_df['Cyclomatic'].mean():.1f}")
print(f"  Total LOC in codebase: {code_metrics_df['CountLineCode'].sum():,}")

# Store key metrics for later analysis
code_summary = {
    'total_files': len(code_metrics_df),
    'total_lines': code_metrics_df['CountLineCode'].sum(),
    'total_classes': code_metrics_df['CountDeclClass'].sum(),
    'total_methods': code_metrics_df['CountDeclMethod'].sum(),
    'avg_complexity': code_metrics_df['Cyclomatic'].mean(),
    'total_units': code_metrics_df['TotalUnits'].sum()
}

print(f"\n✓ Code metrics loaded and processed successfully")

In [ ]:
# Cell [2] - Connect to Neo4j and Retrieve LLM SiFP Estimates
# Purpose: Retrieve LLM-generated SiFP estimates for requirements with established ground truth links
# Dependencies: Neo4j connection, json processing, CONFIG from Cell 0
# Breadcrumbs: Setup -> Code Metrics -> Neo4j Data Retrieval -> LLM Estimates Analysis

def retrieve_llm_estimates():
    """
    Connect to Neo4j and retrieve LLM SiFP estimates for ground truth requirements
    
    Returns:
        tuple: (llm_estimates_df, ground_truth_requirements, model_success_df)
    """
    try:
        # Establish Neo4j connection using configuration
        driver = GraphDatabase.driver(
            CONFIG['NEO4J_URI'], 
            auth=(CONFIG['NEO4J_USER'], CONFIG['NEO4J_PASSWORD'])
        )
        print(f"✓ Connected to Neo4j for project: {CONFIG['NEO4J_PROJECT_NAME']}")

        with driver.session() as session:
            # First, identify TARGET requirements with ground truth
            gt_query = """
                MATCH (r1:Requirement {project: $project_name, type: 'TARGET'})
                WHERE EXISTS((r1)-[:GROUND_TRUTH]-()) OR EXISTS(()-[:GROUND_TRUTH]-(r1))
                RETURN DISTINCT r1.id as requirement_id
            """
            
            gt_result = session.run(gt_query, project_name=CONFIG['NEO4J_PROJECT_NAME'])
            ground_truth_requirements = [record['requirement_id'] for record in gt_result]
            
            print(f"✓ Found {len(ground_truth_requirements)} TARGET requirements with ground truth")

            # Query for LLM estimates on ground truth requirements
            estimation_query = """
                MATCH (r1:Requirement {project: $project_name, type: 'TARGET'})-[se:SIFP_ESTIMATION]->(r2:Requirement)
                WHERE r1.id IN $ground_truth_reqs
                  AND se.final_estimation IS NOT NULL
                  AND se.is_valid = true
                RETURN DISTINCT r1.id as requirement_id,
                       r1.content as requirement_content,
                       se.model as model,
                       se.actor_analysis as actor_json,
                       se.final_estimation as final_json,
                       se.judge_evaluation as judge_eval_json,
                       se.confidence as confidence,
                       se.judge_confidence as judge_confidence,
                       se.judge_score as judge_score
                ORDER BY r1.id, se.model
            """
            
            result = session.run(estimation_query, 
                               project_name=CONFIG['NEO4J_PROJECT_NAME'], 
                               ground_truth_reqs=ground_truth_requirements)
            
            # Process and structure the results
            records = []
            for record in result:
                try:
                    # Parse JSON data from Neo4j
                    actor_data = json.loads(record['actor_json']) if record['actor_json'] else {}
                    final_data = json.loads(record['final_json']) if record['final_json'] else {}
                    
                    # Extract UGEP and UGDG counts
                    actor_ugep = len(actor_data.get('ugeps', []))
                    actor_ugdg = len(actor_data.get('ugdgs', []))
                    final_ugep = len(final_data.get('ugeps', []))
                    final_ugdg = len(final_data.get('ugdgs', []))
                    
                    # Calculate SiFP using standard formula: SiFP = 4.6 × UGEP + 7.0 × UGDG
                    actor_sifp = 4.6 * actor_ugep + 7 * actor_ugdg
                    final_sifp = 4.6 * final_ugep + 7 * final_ugdg
                    
                    records.append({
                        'requirement_id': record['requirement_id'],
                        'requirement_content': record['requirement_content'][:100] + '...',
                        'model': record['model'],
                        'actor_ugep': actor_ugep,
                        'actor_ugdg': actor_ugdg,
                        'actor_sifp': actor_sifp,
                        'final_ugep': final_ugep,
                        'final_ugdg': final_ugdg,
                        'final_sifp': final_sifp,
                        'judge_score': record['judge_score'],
                        'confidence': record['confidence']
                    })
                except Exception as e:
                    print(f"Warning: Error processing record for {record.get('requirement_id', 'unknown')}: {e}")
                    continue

        # Create estimates DataFrame
        llm_estimates_df = pd.DataFrame(records)
        
        if not llm_estimates_df.empty:
            print(f"\n✓ Retrieved {len(llm_estimates_df)} LLM estimates for ground truth requirements")
            
            # Calculate model success rates
            print("\nModel Success Rates (Ground Truth Requirements):")
            print("=" * 50)
            
            model_success = []
            for model in sorted(llm_estimates_df['model'].unique()):
                model_estimates = llm_estimates_df[llm_estimates_df['model'] == model]
                successful_reqs = model_estimates['requirement_id'].nunique()
                success_rate = successful_reqs / len(ground_truth_requirements) * 100
                
                model_success.append({
                    'model': model,
                    'successful_estimates': successful_reqs,
                    'total_ground_truth': len(ground_truth_requirements),
                    'success_rate': success_rate
                })
                
                print(f"  {model}: {successful_reqs}/{len(ground_truth_requirements)} ({success_rate:.1f}%)")
            
            model_success_df = pd.DataFrame(model_success)
            
            # Display sample estimates for verification
            print("\nSample LLM estimates (with ground truth):")
            display_cols = ['requirement_id', 'model', 'final_sifp', 'judge_score']
            print(llm_estimates_df[display_cols].head())
            
            return llm_estimates_df, ground_truth_requirements, model_success_df
            
        else:
            print("Warning: No LLM estimates found for ground truth requirements!")
            return pd.DataFrame(), ground_truth_requirements, pd.DataFrame()
            
    except Exception as e:
        print(f"Error retrieving LLM estimates: {e}")
        raise
    finally:
        driver.close()
        print("✓ Neo4j connection closed")

# Execute the retrieval process
llm_estimates_df, ground_truth_requirements, model_success_df = retrieve_llm_estimates()

In [ ]:
# Cell [3] - Establish Requirements-to-Code Mapping and Feature Analysis
# Purpose: Create mapping between requirements and actual code files for validation baseline
# Dependencies: llm_estimates_df from Cell 2, feature extraction logic
# Breadcrumbs: Setup -> Data Retrieval -> Requirements Mapping -> Feature Analysis

def analyze_requirement_features():
    """
    Analyze requirements by extracting feature identifiers and establishing mappings
    
    Returns:
        tuple: (feature_requirements_df, feature_mapping)
    """
    
    def extract_feature_from_requirement(req_id):
        """
        Extract feature/module name from requirement ID using common patterns
        
        Args:
            req_id (str): Requirement identifier
            
        Returns:
            str: Extracted feature name
        """
        # Handle common requirement ID patterns
        if 'UC' in req_id:
            # Use case format: UC1.1 -> UC1
            return req_id.split('.')[0]
        elif '-' in req_id:
            # Functional requirement format: FR-AUTH-001 -> AUTH
            parts = req_id.split('-')
            if len(parts) >= 2:
                return parts[1]
        return req_id  # Return original if no pattern matches

    # Check if we have LLM estimates to analyze
    if not llm_estimates_df.empty:
        print("Analyzing requirement features and groupings...")
        print("=" * 50)
        
        # Extract features from requirement IDs
        llm_estimates_df['feature'] = llm_estimates_df['requirement_id'].apply(extract_feature_from_requirement)
        
        # Group requirements by feature for analysis
        feature_requirements = llm_estimates_df.groupby('feature').agg({
            'requirement_id': 'nunique',  # Count unique requirements
            'final_sifp': ['mean', 'sum', 'std'],
            'model': 'nunique'  # Count how many models estimated this feature
        }).round(2)
        
        # Flatten column names for better readability
        feature_requirements.columns = [
            'unique_requirements', 'avg_sifp', 'total_sifp', 'std_sifp', 'models_count'
        ]
        
        # Sort by total SiFP for better insights
        feature_requirements = feature_requirements.sort_values('total_sifp', ascending=False)
        
        print("Requirements grouped by feature:")
        print(feature_requirements)
        
        # Calculate feature statistics
        print(f"\nFeature Analysis Summary:")
        print(f"  Total features identified: {len(feature_requirements)}")
        print(f"  Average requirements per feature: {feature_requirements['unique_requirements'].mean():.1f}")
        print(f"  Average SiFP per feature: {feature_requirements['avg_sifp'].mean():.1f}")
        print(f"  Most complex feature: {feature_requirements.index[0]} ({feature_requirements['total_sifp'].max():.1f} SiFP)")
        
        # Create feature mapping for traceability
        feature_mapping = {}
        for feature in feature_requirements.index:
            feature_reqs = llm_estimates_df[llm_estimates_df['feature'] == feature]['requirement_id'].unique()
            feature_mapping[feature] = {
                'requirements': list(feature_reqs),
                'count': len(feature_reqs),
                'estimated_loc': feature_requirements.loc[feature, 'total_sifp'] * CONFIG.get('avg_loc_per_sifp', 100)
            }
        
        return feature_requirements, feature_mapping
        
    else:
        print("Warning: No LLM estimates available for feature analysis")
        return pd.DataFrame(), {}

# Execute feature analysis
if not llm_estimates_df.empty:
    feature_requirements_df, feature_mapping = analyze_requirement_features()
    
    # Display insights about the mapping approach
    print(f"\nRequirement-to-Code Mapping Approach:")
    print("=" * 50)
    print("• Using aggregate analysis based on requirement feature groupings")
    print("• Features extracted from requirement IDs using pattern matching")
    print("• In production, explicit traceability links would provide direct mapping")
    print("• Current approach enables statistical validation at feature level")
    
else:
    print("Skipping feature analysis - no LLM estimates available")
    feature_requirements_df = pd.DataFrame()
    feature_mapping = {}

In [ ]:
# Cell [4] - Calculate Normalized Metrics and Establish Conversion Baselines
# Purpose: Establish normalized relationships between SiFP and code metrics using UFP→SiFP conversion
# Dependencies: code_summary from Cell 1, llm_estimates_df from Cell 2, CONFIG from Cell 0
# Breadcrumbs: Setup -> Data Collection -> Mapping -> Baseline Establishment

def calculate_normalized_metrics():
    """
    Calculate normalized metrics and establish baseline relationships between SiFP and code metrics
    
    Returns:
        tuple: (llm_analysis_df, baseline_metrics, industry_metrics)
    """
    
    print("Code Base Summary (Full Codebase):")
    print("=" * 40)
    for key, value in code_summary.items():
        print(f"  {key}: {value:.0f}")

    # Calculate normalized code metrics
    print("\nNormalized Code Metrics (Full Codebase):")
    print("-" * 40)

    # Key normalized metrics
    loc_per_file = code_summary['total_lines'] / code_summary['total_files']
    methods_per_kloc = (code_summary['total_methods'] / code_summary['total_lines']) * 1000
    classes_per_kloc = (code_summary['total_classes'] / code_summary['total_lines']) * 1000

    print(f"  Lines of code per file: {loc_per_file:.1f}")
    print(f"  Methods per KLOC: {methods_per_kloc:.1f}")
    print(f"  Classes per KLOC: {classes_per_kloc:.1f}")

    # Establish industry baselines
    print("\n" + "="*60)
    print("BASELINE CALCULATION APPROACHES")
    print("="*60)

    # Industry standards from research
    INDUSTRY_LOC_PER_UFP = 100  # Typical for Java
    INDUSTRY_LOC_PER_SIFP = INDUSTRY_LOC_PER_UFP / CONFIG['CONVERSION_FACTOR']  # Adjust for conversion

    industry_metrics = {
        'LOC_PER_UFP': INDUSTRY_LOC_PER_UFP,
        'LOC_PER_SIFP': INDUSTRY_LOC_PER_SIFP,
        'SIFP_PER_KLOC': 1000/INDUSTRY_LOC_PER_SIFP
    }

    print(f"\nIndustry Baseline (Research-based):")
    print(f"  Typical LOC per UFP (Java): {INDUSTRY_LOC_PER_UFP}")
    print(f"  Implied LOC per SiFP: {INDUSTRY_LOC_PER_SIFP:.1f}")
    print(f"  SiFP per KLOC: {industry_metrics['SIFP_PER_KLOC']:.2f}")

    # Analyze LLM estimates if available
    if not llm_estimates_df.empty and len(ground_truth_requirements) > 0:
        print("\n" + "="*60)
        print("LLM SIFP ANALYSIS (Scaled to Estimated Requirements)")
        print("="*60)
        
        # Get unique requirements that were successfully estimated
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        estimation_coverage = len(estimated_requirements) / len(ground_truth_requirements)
        
        print(f"\nEstimation Coverage:")
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
        print(f"  Requirements with estimates: {len(estimated_requirements)} ({estimation_coverage:.1%})")
        
        # Scale code metrics based on estimation coverage
        scaled_code_metrics = {
            'lines': code_summary['total_lines'] * estimation_coverage,
            'classes': code_summary['total_classes'] * estimation_coverage,
            'methods': code_summary['total_methods'] * estimation_coverage
        }
        
        print(f"\nScaled Code Metrics (for estimated requirements only):")
        print(f"  Estimated lines of code: {scaled_code_metrics['lines']:.0f}")
        print(f"  Estimated classes: {scaled_code_metrics['classes']:.0f}")
        print(f"  Estimated methods: {scaled_code_metrics['methods']:.0f}")
        
        # Calculate metrics for each LLM model
        llm_analysis = []
        
        for model in sorted(llm_estimates_df['model'].unique()):
            model_data = llm_estimates_df[llm_estimates_df['model'] == model]
            
            # Calculate model totals
            total_sifp = model_data['final_sifp'].sum()
            total_ugep = model_data['final_ugep'].sum()
            total_ugdg = model_data['final_ugdg'].sum()
            successful_reqs = model_data['requirement_id'].nunique()
            
            # Calculate normalized metrics based on MODEL-SPECIFIC coverage
            model_coverage = successful_reqs / len(ground_truth_requirements)
            model_estimated_loc = code_summary['total_lines'] * model_coverage
            
            # Key normalized metrics
            sifp_per_kloc = (total_sifp / model_estimated_loc) * 1000 if model_estimated_loc > 0 else 0
            loc_per_sifp = model_estimated_loc / total_sifp if total_sifp > 0 else 0
            sifp_per_req = total_sifp / successful_reqs if successful_reqs > 0 else 0
            
            # Calculate equivalent UFP for comparison
            equivalent_ufp = total_sifp / CONFIG['CONVERSION_FACTOR']
            
            llm_analysis.append({
                'model': model,
                'successful_reqs': successful_reqs,
                'coverage': model_coverage,
                'total_sifp': total_sifp,
                'equivalent_ufp': equivalent_ufp,
                'total_ugep': total_ugep,
                'total_ugdg': total_ugdg,
                'estimated_loc': model_estimated_loc,
                'sifp_per_kloc': sifp_per_kloc,
                'loc_per_sifp': loc_per_sifp,
                'sifp_per_req': sifp_per_req
            })
            
            print(f"\n{model}:")
            print(f"  Successfully estimated: {successful_reqs}/{len(ground_truth_requirements)} requirements ({model_coverage:.1%})")
            print(f"  Total SiFP: {total_sifp:.1f} (equivalent to {equivalent_ufp:.1f} UFP)")
            print(f"  Estimated LOC coverage: {model_estimated_loc:.0f} lines")
            print(f"  SiFP per KLOC: {sifp_per_kloc:.2f}")
            print(f"  LOC per SiFP point: {loc_per_sifp:.1f}")
            print(f"  Deviation from industry baseline: {(loc_per_sifp - INDUSTRY_LOC_PER_SIFP)/INDUSTRY_LOC_PER_SIFP*100:+.1f}%")
        
        # Create analysis DataFrame
        llm_analysis_df = pd.DataFrame(llm_analysis)
        
        # Calculate project-specific baseline (weighted average of LLM estimates)
        if not llm_analysis_df.empty:
            weighted_loc_per_sifp = np.average(llm_analysis_df['loc_per_sifp'], 
                                              weights=llm_analysis_df['coverage'])
            
            baseline_metrics = {
                'project_loc_per_sifp': weighted_loc_per_sifp,
                'industry_loc_per_sifp': INDUSTRY_LOC_PER_SIFP,
                'difference_pct': (weighted_loc_per_sifp - INDUSTRY_LOC_PER_SIFP)/INDUSTRY_LOC_PER_SIFP*100,
                'estimated_requirements': estimated_requirements,
                'scaled_code_metrics': scaled_code_metrics
            }
            
            print("\n\nBASELINE COMPARISON:")
            print("-" * 40)
            print(f"  Industry baseline LOC/SiFP: {INDUSTRY_LOC_PER_SIFP:.1f}")
            print(f"  Project baseline LOC/SiFP (weighted avg): {weighted_loc_per_sifp:.1f}")
            print(f"  Difference: {baseline_metrics['difference_pct']:+.1f}%")
            
            return llm_analysis_df, baseline_metrics, industry_metrics
        
    # Return empty results if no LLM data
    return pd.DataFrame(), {}, industry_metrics

# Execute the normalized metrics calculation
llm_analysis_df, baseline_metrics, industry_metrics = calculate_normalized_metrics()

print(f"\n✓ Normalized metrics calculated successfully")

In [ ]:
# Cell [5] - Detailed Normalized Performance Analysis
# Purpose: Analyze model accuracy in normalized units (per SiFP point) with quality metrics
# Dependencies: llm_analysis_df and baseline_metrics from Cell 4, llm_estimates_df from Cell 2  
# Breadcrumbs: Setup -> Data Collection -> Baseline Establishment -> Performance Analysis

def analyze_model_performance():
    """
    Analyze detailed performance metrics for each LLM model
    
    Returns:
        pd.DataFrame: Performance analysis with accuracy rankings
    """
    
    if llm_estimates_df.empty or llm_analysis_df.empty:
        print("Warning: No LLM data available for performance analysis")
        return pd.DataFrame()
    
    print("Normalized Model Performance Analysis (Per SiFP Point)")
    print("=" * 60)
    
    model_performance = []
    
    # Use the project baseline from calculated metrics
    baseline_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', 
                                                industry_metrics.get('LOC_PER_SIFP', 100))
    
    print(f"Using baseline: {baseline_loc_per_sifp:.1f} LOC per SiFP")
    print("-" * 60)
    
    for _, row in llm_analysis_df.iterrows():
        model = row['model']
        
        # Calculate normalized accuracy metrics
        loc_per_sifp_error = row['loc_per_sifp'] - baseline_loc_per_sifp
        loc_per_sifp_error_pct = (loc_per_sifp_error / baseline_loc_per_sifp) * 100 if baseline_loc_per_sifp > 0 else 0
        
        # Get quality metrics from original LLM data
        model_data = llm_estimates_df[llm_estimates_df['model'] == model]
        avg_confidence = model_data['confidence'].mean() if 'confidence' in model_data.columns else 0
        avg_judge_score = model_data['judge_score'].mean() if 'judge_score' in model_data.columns else 0
        std_sifp = model_data['final_sifp'].std() if 'final_sifp' in model_data.columns else 0
        
        # Calculate success rate
        success_rate = row['successful_reqs'] / len(ground_truth_requirements) if len(ground_truth_requirements) > 0 else 0
        
        model_performance.append({
            'Model': model,
            'Success_Rate': success_rate,
            'SiFP_per_KLOC': row['sifp_per_kloc'],
            'LOC_per_SiFP': row['loc_per_sifp'],
            'LOC_per_SiFP_Error': loc_per_sifp_error,
            'Error_Pct': abs(loc_per_sifp_error_pct),
            'Avg_SiFP_per_Req': row['sifp_per_req'],
            'Std_SiFP': std_sifp,
            'Avg_Confidence': avg_confidence,
            'Avg_Judge_Score': avg_judge_score
        })
        
        # Display individual model analysis
        print(f"\n{model}:")
        print(f"  Success rate: {success_rate:.1%}")
        print(f"  SiFP per KLOC: {row['sifp_per_kloc']:.2f}")
        print(f"  LOC per SiFP point: {row['loc_per_sifp']:.1f}")
        print(f"  Error vs baseline: {loc_per_sifp_error:+.1f} LOC/SiFP ({loc_per_sifp_error_pct:+.1f}%)")
        print(f"  Average confidence: {avg_confidence:.2%}")
        print(f"  Average judge score: {avg_judge_score:.2f}/5")
        print(f"  SiFP variability (std): {std_sifp:.2f}")
    
    # Create performance DataFrame
    performance_df = pd.DataFrame(model_performance)
    
    if not performance_df.empty:
        # Rank models by normalized accuracy (lower error is better)
        performance_df['Accuracy_Rank'] = performance_df['Error_Pct'].rank()
        
        print("\n\nNormalized Performance Summary:")
        print("=" * 60)
        summary_cols = ['Model', 'Success_Rate', 'LOC_per_SiFP', 'Error_Pct', 'Accuracy_Rank']
        print(performance_df[summary_cols].round(3).to_string(index=False))
        
        # Additional insights
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]
        best_coverage = performance_df.loc[performance_df['Success_Rate'].idxmax()]
        
        print(f"\nKey Insights:")
        print(f"  Most accurate model: {best_accuracy['Model']} ({best_accuracy['Error_Pct']:.1f}% error)")
        print(f"  Best coverage model: {best_coverage['Model']} ({best_coverage['Success_Rate']:.1%} success rate)")
        print(f"  Average error across all models: {performance_df['Error_Pct'].mean():.1f}%")
        
    return performance_df

# Execute performance analysis
performance_df = analyze_model_performance()

print(f"\n✓ Performance analysis completed successfully")

In [ ]:
# Cell [6] - Load Desharnais Dataset and Establish UFP→SiFP→Effort Relationships  
# Purpose: Load industry benchmark dataset and establish the complete conversion chain for effort estimation
# Dependencies: sklearn LinearRegression, CONFIG from Cell 0, pandas processing
# Breadcrumbs: Setup -> Performance Analysis -> Industry Benchmarks -> Effort Conversion Chain

def load_and_analyze_desharnais():
    """
    Load Desharnais dataset and establish UFP→SiFP→Effort conversion relationships
    
    Returns:
        tuple: (desharnais_df, effort_metrics, effort_model)
    """
    try:
        # Load industry benchmark dataset
        desharnais_df = pd.read_csv('../datasets/CostEstimation/Desharnais.csv')
        print(f"✓ Loaded Desharnais dataset: {desharnais_df.shape[0]} projects")

        # Identify column names (handle variations in dataset)
        ufp_column = 'PointsNonAdjust' if 'PointsNonAdjust' in desharnais_df.columns else 'UFP'
        effort_column = 'Effort' if 'Effort' in desharnais_df.columns else 'effort'

        print(f"Using columns: UFP='{ufp_column}', Effort='{effort_column}'")
        
        # Apply UFP to SiFP conversion using research-validated factor
        print(f"\nApplying UFP→SiFP conversion factor: {CONFIG['CONVERSION_FACTOR']}")
        desharnais_df['SiFP_converted'] = desharnais_df[ufp_column] * CONFIG['CONVERSION_FACTOR']

        # Calculate effort per SiFP metrics
        print("\nDesharnais Normalized Metrics:")
        print("=" * 40)

        # Calculate hours per SiFP point for each project
        desharnais_df['hours_per_sifp'] = desharnais_df[effort_column] / desharnais_df['SiFP_converted']

        # Calculate summary statistics
        effort_metrics = {
            'avg_hours_per_sifp': desharnais_df['hours_per_sifp'].mean(),
            'median_hours_per_sifp': desharnais_df['hours_per_sifp'].median(),
            'std_hours_per_sifp': desharnais_df['hours_per_sifp'].std(),
            'min_hours_per_sifp': desharnais_df['hours_per_sifp'].min(),
            'max_hours_per_sifp': desharnais_df['hours_per_sifp'].max()
        }

        print(f"  Average hours per SiFP: {effort_metrics['avg_hours_per_sifp']:.2f}")
        print(f"  Median hours per SiFP: {effort_metrics['median_hours_per_sifp']:.2f}")
        print(f"  Std dev hours per SiFP: {effort_metrics['std_hours_per_sifp']:.2f}")
        print(f"  Range: {effort_metrics['min_hours_per_sifp']:.2f} - {effort_metrics['max_hours_per_sifp']:.2f}")

        # Build linear effort prediction model
        print(f"\nBuilding Linear Effort Model:")
        print("-" * 30)
        
        # Prepare data for sklearn
        X = desharnais_df[['SiFP_converted']].values.astype(np.float64)
        y = desharnais_df[effort_column].values.astype(np.float64)

        # Fit linear regression model
        effort_model = LinearRegression()
        effort_model.fit(X, y)

        # Extract model coefficients
        linear_hours_per_sifp = float(effort_model.coef_[0])
        intercept = float(effort_model.intercept_)
        
        # Calculate model performance
        y_pred = effort_model.predict(X)
        r2 = float(r2_score(y, y_pred))

        print(f"  Hours per SiFP (coefficient): {linear_hours_per_sifp:.2f}")
        print(f"  Base hours (intercept): {intercept:.2f}")
        print(f"  R² score: {r2:.3f}")
        
        # Add model metrics to effort_metrics
        effort_metrics.update({
            'linear_hours_per_sifp': linear_hours_per_sifp,
            'intercept': intercept,
            'r2_score': r2
        })

        # Analyze SiFP distribution in industry data
        print(f"\nSiFP Distribution in Desharnais Dataset:")
        print("-" * 40)
        print(f"  Mean SiFP per project: {desharnais_df['SiFP_converted'].mean():.1f}")
        print(f"  Median SiFP per project: {desharnais_df['SiFP_converted'].median():.1f}")
        print(f"  Range: {desharnais_df['SiFP_converted'].min():.1f} - {desharnais_df['SiFP_converted'].max():.1f}")
        print(f"  Total projects: {len(desharnais_df)}")

        return desharnais_df, effort_metrics, effort_model
        
    except Exception as e:
        print(f"Error loading Desharnais dataset: {e}")
        raise

# Execute Desharnais analysis
desharnais_df, effort_metrics, effort_model = load_and_analyze_desharnais()

print(f"\n✓ Desharnais dataset analysis completed successfully")

In [ ]:
# Cell [7] - Normalized Effort Impact Analysis with Complete Conversion Chain
# Purpose: Analyze effort impact using UFP→SiFP→Effort conversion chain and calculate cost implications  
# Dependencies: performance_df from Cell 5, effort_metrics from Cell 6, CONFIG from Cell 0
# Breadcrumbs: Setup -> Performance Analysis -> Industry Benchmarks -> Effort Impact Analysis

def analyze_effort_impact():
    """
    Analyze effort impact using the complete UFP→SiFP→Effort conversion chain
    
    Returns:
        pd.DataFrame: Effort impact analysis with cost implications
    """
    
    if performance_df.empty or not effort_metrics:
        print("Warning: Missing required data for effort impact analysis")
        return pd.DataFrame()
    
    print("Normalized Effort Impact Analysis (Per SiFP Point)")
    print("=" * 60)

    # Display conversion chain information
    print(f"\nConversion Chain:")
    print(f"  UFP → SiFP: Factor = {CONFIG['CONVERSION_FACTOR']} (SiFP = {CONFIG['CONVERSION_FACTOR']} × UFP)")
    print(f"  SiFP → Effort: {effort_metrics['avg_hours_per_sifp']:.2f} hours/SiFP (from Desharnais)")
    
    if baseline_metrics:
        print(f"  SiFP → LOC: {baseline_metrics['project_loc_per_sifp']:.1f} LOC/SiFP (project baseline)")
    
    effort_impact = []
    
    for _, row in performance_df.iterrows():
        model = row['Model']
        
        # Get model's LOC per SiFP
        model_loc_per_sifp = row['LOC_per_SiFP']
        baseline_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', 
                                                   industry_metrics.get('LOC_PER_SIFP', 100))
        
        # Calculate SiFP estimation accuracy
        # If model estimates fewer LOC per SiFP, it's overestimating SiFP count
        sifp_estimation_factor = baseline_loc_per_sifp / model_loc_per_sifp if model_loc_per_sifp > 0 else 1
        
        # Calculate effort impact using Desharnais baseline
        baseline_hours_per_sifp = effort_metrics['avg_hours_per_sifp']
        
        # The effective hours per estimated SiFP
        effective_hours_per_estimated_sifp = baseline_hours_per_sifp / sifp_estimation_factor
        
        # Calculate percentage error in effort estimation
        effort_error_pct = (sifp_estimation_factor - 1) * 100
        
        # Get total SiFP estimated by this model
        if not llm_analysis_df.empty:
            model_row = llm_analysis_df[llm_analysis_df['model'] == model]
            if not model_row.empty:
                model_total_sifp = model_row['total_sifp'].values[0]
                actual_sifp = model_total_sifp / sifp_estimation_factor
                
                # Calculate total effort impact
                estimated_total_effort = model_total_sifp * baseline_hours_per_sifp
                actual_total_effort = actual_sifp * baseline_hours_per_sifp
                total_effort_error = estimated_total_effort - actual_total_effort
                
                # Calculate cost impact using standard rate
                total_cost_impact = total_effort_error * CONFIG['COST_PER_HOUR']
            else:
                model_total_sifp = actual_sifp = total_effort_error = total_cost_impact = 0
        else:
            model_total_sifp = actual_sifp = total_effort_error = total_cost_impact = 0
        
        effort_impact.append({
            'Model': model,
            'LOC_per_SiFP': model_loc_per_sifp,
            'SiFP_Estimation_Factor': sifp_estimation_factor,
            'Desharnais_Hours_per_SiFP': baseline_hours_per_sifp,
            'Effective_Hours_per_Est_SiFP': effective_hours_per_estimated_sifp,
            'Effort_Error_Pct': effort_error_pct,
            'Model_Total_SiFP': model_total_sifp,
            'Actual_SiFP': actual_sifp,
            'Total_Effort_Error_Hours': total_effort_error,
            'Total_Cost_Impact_USD': total_cost_impact
        })
        
        # Display model-specific analysis
        print(f"\n{model}:")
        print(f"  LOC per SiFP: {model_loc_per_sifp:.1f} (baseline: {baseline_loc_per_sifp:.1f})")
        print(f"  SiFP estimation factor: {sifp_estimation_factor:.2f}x")
        print(f"  Interpretation: Model {'overestimates' if sifp_estimation_factor > 1 else 'underestimates'} SiFP count")
        print(f"  Desharnais baseline: {baseline_hours_per_sifp:.2f} hours per actual SiFP")
        print(f"  Effective hours per estimated SiFP: {effective_hours_per_estimated_sifp:.2f}")
        print(f"  Effort estimation error: {effort_error_pct:+.1f}%")
        if model_total_sifp > 0:
            print(f"  Total SiFP estimated: {model_total_sifp:.0f}")
            print(f"  Actual SiFP (implied): {actual_sifp:.0f}")
            print(f"  Total effort error: {total_effort_error:+.0f} hours (${total_cost_impact:+,.0f})")
    
    effort_impact_df = pd.DataFrame(effort_impact)
    
    if not effort_impact_df.empty:
        # Summary statistics
        print("\n\nEffort Impact Summary:")
        print("=" * 40)
        print(f"  Desharnais hours per SiFP: {effort_metrics['avg_hours_per_sifp']:.2f}")
        print(f"  Average SiFP estimation factor: {effort_impact_df['SiFP_Estimation_Factor'].mean():.2f}x")
        print(f"  Average effort error: {effort_impact_df['Effort_Error_Pct'].mean():+.1f}%")
        print(f"  Total cost impact range: ${effort_impact_df['Total_Cost_Impact_USD'].min():,.0f} to ${effort_impact_df['Total_Cost_Impact_USD'].max():,.0f}")
        
        if effort_impact_df['Effort_Error_Pct'].abs().size > 0:
            best_model = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin(), 'Model']
            print(f"  Most accurate effort model: {best_model}")
    
    return effort_impact_df

# Execute effort impact analysis
effort_impact_df = analyze_effort_impact()

print(f"\n✓ Effort impact analysis completed successfully")

In [ ]:
# Cell [8] - Comprehensive Visualization of Normalized Results and Distributions
# Purpose: Create comprehensive visualizations of model performance, accuracy distributions, and cost impacts
# Dependencies: performance_df from Cell 5, effort_impact_df from Cell 7, matplotlib/seaborn from Cell 0
# Breadcrumbs: Setup -> Performance Analysis -> Effort Impact -> Comprehensive Visualization

if 'performance_df' in globals() and 'effort_impact_df' in globals() and not performance_df.empty and not effort_impact_df.empty:
    
    # Get baseline values from previous calculations
    avg_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_metrics.get('LOC_PER_SIFP', 100))
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10)
    project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project')
    
    # Create a larger figure with more subplots
    fig = plt.figure(figsize=(20, 16))
    
    # Define grid for subplots
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)
    
    models = performance_df['Model'].values
    x = np.arange(len(models))
    
    # 1. LOC per SiFP Point Comparison
    ax1 = fig.add_subplot(gs[0, :2])
    bars = ax1.bar(x, performance_df['LOC_per_SiFP'], alpha=0.7, color='skyblue')
    ax1.axhline(y=avg_loc_per_sifp, color='red', linestyle='--', 
                label=f'Baseline ({avg_loc_per_sifp:.1f} LOC/SiFP)')
    
    # Color bars based on performance
    for i, bar in enumerate(bars):
        if performance_df.iloc[i]['LOC_per_SiFP'] < avg_loc_per_sifp * 0.8:
            bar.set_color('green')
        elif performance_df.iloc[i]['LOC_per_SiFP'] > avg_loc_per_sifp * 1.2:
            bar.set_color('red')
    
    ax1.set_xlabel('Model')
    ax1.set_ylabel('Lines of Code per SiFP Point')
    ax1.set_title('Code Density per SiFP Point by Model')
    ax1.set_xticks(x)
    ax1.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add success rate and error annotations
    for i, (model, success_rate, error) in enumerate(zip(models, performance_df['Success_Rate'], performance_df['Error_Pct'])):
        ax1.text(i, performance_df.iloc[i]['LOC_per_SiFP'] + 1, 
                f'{success_rate:.0%}\n±{error:.0f}%', ha='center', va='bottom', fontsize=8)
    
    # 2. Model Performance Comparison Table
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.axis('tight')
    ax2.axis('off')
    
    # Create performance summary table
    table_data = []
    for _, row in performance_df.iterrows():
        model_name = row['Model'].split('/')[-1][:20]
        table_data.append([
            model_name,
            f"{row['Success_Rate']:.0%}",
            f"{row['LOC_per_SiFP']:.1f}",
            f"{row['Error_Pct']:.0f}%"
        ])
    
    table = ax2.table(cellText=table_data,
                     colLabels=['Model', 'Success', 'LOC/SiFP', 'Error'],
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.5)
    ax2.set_title('Model Performance Summary', pad=20)
    
    # 3. Effort per SiFP Point
    ax3 = fig.add_subplot(gs[1, 0])
    bars = ax3.bar(x, effort_impact_df['Effective_Hours_per_Est_SiFP'], 
            color=['green' if x < desharnais_hours_per_sifp else 'orange' 
                   for x in effort_impact_df['Effective_Hours_per_Est_SiFP']], alpha=0.7)
    ax3.axhline(y=desharnais_hours_per_sifp, color='red', linestyle='--', 
                label=f'Desharnais Baseline ({desharnais_hours_per_sifp:.1f} hrs/SiFP)')
    ax3.set_xlabel('Model')
    ax3.set_ylabel('Effective Hours per Estimated SiFP')
    ax3.set_title('Effort Estimation per SiFP Point')
    ax3.set_xticks(x)
    ax3.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. SiFP per KLOC
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.bar(x, performance_df['SiFP_per_KLOC'], alpha=0.7, color='coral')
    ax4.set_xlabel('Model')
    ax4.set_ylabel('SiFP per KLOC')
    ax4.set_title('Function Point Density (SiFP per 1000 LOC)')
    ax4.set_xticks(x)
    ax4.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax4.grid(True, alpha=0.3)
    
    # 5. Total Cost Impact
    ax5 = fig.add_subplot(gs[1, 2])
    bars = ax5.bar(x, effort_impact_df['Total_Cost_Impact_USD'], 
                   color=['darkgreen' if x < 0 else 'darkred' 
                          for x in effort_impact_df['Total_Cost_Impact_USD']], alpha=0.7)
    ax5.set_xlabel('Model')
    ax5.set_ylabel('Total Cost Impact ($)')
    ax5.set_title('Total Cost Impact')
    ax5.set_xticks(x)
    ax5.set_xticklabels([m.split('/')[-1][:15] for m in models], rotation=45, ha='right')
    ax5.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax5.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(effort_impact_df['Total_Cost_Impact_USD']):
        ax5.text(i, v + (1000 if v > 0 else -1000), f'${v:,.0f}', 
                ha='center', va='bottom' if v > 0 else 'top', fontsize=8)
    
    # 6-10. Histograms for each model showing requirement-level accuracy
    histogram_count = 0
    max_histograms = 6  # Limit to 6 histograms to fit in remaining subplot space
    
    for idx, model in enumerate(models[:max_histograms]):
        row_idx = 2 + histogram_count // 3
        col_idx = histogram_count % 3
        
        if row_idx >= 4:  # Don't exceed our grid
            break
            
        ax = fig.add_subplot(gs[row_idx, col_idx])
        
        model_data = llm_estimates_df[llm_estimates_df['model'] == model]
        
        if not model_data.empty:
            model_coverage = model_data['requirement_id'].nunique() / len(ground_truth_requirements)
            
            # Calculate LOC per SiFP for each requirement
            req_loc_per_sifp = []
            for _, req in model_data.iterrows():
                if req['final_sifp'] > 0:
                    est_loc_per_req = (code_summary['total_lines'] * model_coverage) / model_data['requirement_id'].nunique()
                    loc_per_sifp = est_loc_per_req / req['final_sifp']
                    req_loc_per_sifp.append(loc_per_sifp)
            
            if req_loc_per_sifp:
                # Create histogram
                n, bins, patches = ax.hist(req_loc_per_sifp, bins=min(15, len(req_loc_per_sifp)), 
                                         alpha=0.7, color='steelblue', edgecolor='black')
                
                # Color code bins
                for i, patch in enumerate(patches):
                    if i < len(bins) - 1:  # bins has one more element than patches
                        if bins[i] < avg_loc_per_sifp * 0.8:
                            patch.set_facecolor('green')
                        elif bins[i] > avg_loc_per_sifp * 1.2:
                            patch.set_facecolor('red')
                
                # Add baseline line
                ax.axvline(x=avg_loc_per_sifp, color='red', linestyle='--', linewidth=2, 
                          label=f'Baseline: {avg_loc_per_sifp:.1f}')
                ax.axvline(x=np.mean(req_loc_per_sifp), color='blue', linestyle='-', linewidth=2,
                          label=f'Model mean: {np.mean(req_loc_per_sifp):.1f}')
                
                ax.set_xlabel('LOC per SiFP')
                ax.set_ylabel('# Requirements')
                ax.set_title(f'{model.split("/")[-1][:20]}\nAccuracy Distribution')
                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)
                
                # Add statistics text
                ax.text(0.95, 0.95, f'n={len(req_loc_per_sifp)}\nσ={np.std(req_loc_per_sifp):.1f}',
                       transform=ax.transAxes, ha='right', va='top', fontsize=8,
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            else:
                ax.text(0.5, 0.5, 'No valid data', transform=ax.transAxes, ha='center', va='center')
                ax.set_title(f'{model.split("/")[-1][:20]}\nNo Data')
        else:
            ax.text(0.5, 0.5, 'No model data', transform=ax.transAxes, ha='center', va='center')
            ax.set_title(f'{model.split("/")[-1][:20]}\nNo Data')
        
        histogram_count += 1
    
    plt.suptitle(f'Comprehensive SiFP Analysis - {project_name}\n'
                 f'All Models Performance and Accuracy Distribution', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    print("✓ Comprehensive visualization completed successfully")
    
else:
    print("Cannot create comprehensive visualization - required data not available")
    print("Available variables:")
    if 'performance_df' in globals():
        print(f"  - performance_df: {len(performance_df) if not performance_df.empty else 'empty'}")
    else:
        print("  - performance_df: not defined")
    
    if 'effort_impact_df' in globals():
        print(f"  - effort_impact_df: {len(effort_impact_df) if not effort_impact_df.empty else 'empty'}")
    else:
        print("  - effort_impact_df: not defined")
    
    if 'baseline_metrics' in globals():
        print("  - baseline_metrics: available")
    else:
        print("  - baseline_metrics: not defined")

In [ ]:
# Cell [10] - Executive Summary with Complete UFP→SiFP→LOC→Effort Analysis  
# Purpose: Generate comprehensive executive summary with complete conversion chain analysis and business insights
# Dependencies: All previous analysis results, CONFIG settings, comprehensive metrics from entire workflow
# Breadcrumbs: Setup -> Analysis -> Recommendations -> Executive Summary & Business Impact Report

print("EXECUTIVE SUMMARY - NORMALIZED SIFP ANALYSIS")
print("=" * 60)

# Get project name safely
project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project') if 'CONFIG' in globals() else 'Unknown Project'
print(f"Project: {project_name}")
print(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")

if 'performance_df' in globals() and 'effort_impact_df' in globals() and not performance_df.empty and not effort_impact_df.empty:
    print(f"\nData Summary:")
    
    if 'code_metrics_df' in globals():
        print(f"  Total code files in project: {len(code_metrics_df)}")
        print(f"  Total lines of code in project: {code_metrics_df['CountLineCode'].sum():,}")
    else:
        print("  Code metrics: Not available")
    
    if 'ground_truth_requirements' in globals():
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
    else:
        print("  Ground truth requirements: Not available")
    
    if 'llm_estimates_df' in globals() and not llm_estimates_df.empty:
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        if 'ground_truth_requirements' in globals():
            print(f"  Requirements with estimates: {len(estimated_requirements)} ({len(estimated_requirements)/len(ground_truth_requirements):.1%})")
        else:
            print(f"  Requirements with estimates: {len(estimated_requirements)}")
    
    # Get baseline values safely
    conversion_factor = CONFIG.get('CONVERSION_FACTOR', 0.957) if 'CONFIG' in globals() else 0.957
    
    # Get industry and project baselines
    industry_loc_per_sifp = industry_metrics.get('LOC_PER_SIFP', 100) if 'industry_metrics' in globals() else 100
    project_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_loc_per_sifp) if 'baseline_metrics' in globals() else industry_loc_per_sifp
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10) if 'effort_metrics' in globals() else 10
    
    print(f"\nConversion Factors and Baselines:")
    print(f"  UFP → SiFP: {conversion_factor} (from Desharnais research)")
    print(f"  SiFP → Effort: {desharnais_hours_per_sifp:.2f} hours/SiFP (Desharnais dataset)")
    print(f"  SiFP → LOC: {project_loc_per_sifp:.1f} LOC/SiFP (project weighted average)")
    print(f"  Industry baseline: {industry_loc_per_sifp:.1f} LOC/SiFP")
    print(f"  Project vs Industry: {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}%")
    
    # Detailed performance for each model
    print(f"\n" + "="*80)
    print("DETAILED MODEL PERFORMANCE")
    print("="*80)
    
    for idx, row in performance_df.iterrows():
        model = row['Model']
        effort_row = effort_impact_df[effort_impact_df['Model'] == model]
        
        if not effort_row.empty:
            effort_row = effort_row.iloc[0]
            
            if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
                llm_row = llm_analysis_df[llm_analysis_df['model'] == model]
                if not llm_row.empty:
                    llm_row = llm_row.iloc[0]
                else:
                    llm_row = None
            else:
                llm_row = None
            
            print(f"\n{idx+1}. {model}")
            print("-" * len(f"{idx+1}. {model}"))
            
            print(f"\n  Estimation Coverage:")
            print(f"    - Success rate: {row['Success_Rate']:.1%}")
            if llm_row is not None:
                print(f"    - Requirements estimated: {llm_row['successful_reqs']}/{len(ground_truth_requirements) if 'ground_truth_requirements' in globals() else 'unknown'}")
            
            print(f"\n  SiFP Estimates:")
            if llm_row is not None:
                print(f"    - Total SiFP: {llm_row['total_sifp']:.0f}")
                print(f"    - Equivalent UFP: {llm_row['equivalent_ufp']:.0f}")
                print(f"    - Average SiFP per requirement: {row['Avg_SiFP_per_Req']:.1f}")
            else:
                print(f"    - Average SiFP per requirement: {row.get('Avg_SiFP_per_Req', 'N/A')}")
            
            print(f"\n  Accuracy Metrics:")
            print(f"    - LOC per SiFP: {row['LOC_per_SiFP']:.1f} (baseline: {project_loc_per_sifp:.1f})")
            print(f"    - Error: {row.get('LOC_per_SiFP_Error', 'N/A'):+.1f} LOC/SiFP ({row['Error_Pct']:+.1f}%)")
            print(f"    - SiFP estimation factor: {effort_row['SiFP_Estimation_Factor']:.2f}x")
            
            print(f"\n  Effort Impact:")
            print(f"    - Desharnais baseline: {desharnais_hours_per_sifp:.2f} hours/SiFP")
            print(f"    - Effort estimation error: {effort_row['Effort_Error_Pct']:+.1f}%")
            print(f"    - Total effort error: {effort_row['Total_Effort_Error_Hours']:+.0f} hours")
            print(f"    - Cost impact: ${effort_row['Total_Cost_Impact_USD']:+,.0f}")
            
            print(f"\n  Quality Indicators:")
            print(f"    - Average confidence: {row.get('Avg_Confidence', 0):.1%}")
            print(f"    - Average judge score: {row.get('Avg_Judge_Score', 0):.2f}/5")
    
    # Analysis of the conversion chain
    print(f"\n" + "="*80)
    print("CONVERSION CHAIN ANALYSIS")
    print("="*80)
    
    print(f"\nFor a typical requirement in this project:")
    if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
        avg_sifp_per_req = llm_analysis_df['sifp_per_req'].mean()
        print(f"  Average SiFP per requirement: {avg_sifp_per_req:.1f}")
        print(f"  Equivalent UFP: {avg_sifp_per_req / conversion_factor:.1f}")
        print(f"  Expected LOC: {avg_sifp_per_req * project_loc_per_sifp:.0f}")
        print(f"  Expected effort: {avg_sifp_per_req * desharnais_hours_per_sifp:.0f} hours")
    else:
        print("  Analysis not available - LLM analysis data missing")
    
    # Recommendations
    print(f"\n" + "="*80)
    print("RECOMMENDATIONS")
    print("="*80)
    
    if not performance_df.empty:
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]['Model']
        print(f"\n1. For most accurate code size estimation: {best_accuracy}")
    else:
        print("\n1. For most accurate code size estimation: Data not available")
    
    if not effort_impact_df.empty:
        best_effort = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin()]['Model']
        print(f"2. For most accurate effort estimation: {best_effort}")
    else:
        print("2. For most accurate effort estimation: Data not available")
    
    print(f"3. Use Desharnais baseline of {desharnais_hours_per_sifp:.1f} hours per SiFP for effort planning")
    print(f"4. Apply UFP conversion factor of {conversion_factor} when comparing to UFP-based estimates")
    print(f"5. Consider that this project has {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}% different LOC/SiFP than industry average")
    
    # Save results with all conversion factors
    try:
        os.makedirs('results', exist_ok=True)
        
        # Create comprehensive summary
        conversion_summary = pd.DataFrame({
            'Metric': ['UFP→SiFP Factor', 'Industry LOC/SiFP', 'Project LOC/SiFP', 'Desharnais Hours/SiFP'],
            'Value': [conversion_factor, industry_loc_per_sifp, project_loc_per_sifp, desharnais_hours_per_sifp]
        })
        conversion_summary.to_csv(f'results/conversion_factors_{project_name}.csv', index=False)
        
        # Save all other results
        performance_df.to_csv(f'results/normalized_performance_{project_name}.csv', index=False)
        effort_impact_df.to_csv(f'results/normalized_effort_impact_{project_name}.csv', index=False)
        
        if 'llm_analysis_df' in globals() and not llm_analysis_df.empty:
            llm_analysis_df.to_csv(f'results/normalized_llm_analysis_{project_name}.csv', index=False)
        
        print(f"\n✓ Results saved to results/ directory")
        
    except Exception as e:
        print(f"\nWarning: Could not save results - {e}")

else:
    print("Missing required data for executive summary")
    print("Available data:")
    
    if 'performance_df' in globals():
        print(f"  - performance_df: {len(performance_df) if not performance_df.empty else 'empty'}")
    else:
        print("  - performance_df: not available")
    
    if 'effort_impact_df' in globals():
        print(f"  - effort_impact_df: {len(effort_impact_df) if not effort_impact_df.empty else 'empty'}")
    else:
        print("  - effort_impact_df: not available")
    
    if 'llm_analysis_df' in globals():
        print(f"  - llm_analysis_df: {len(llm_analysis_df) if not llm_analysis_df.empty else 'empty'}")
    else:
        print("  - llm_analysis_df: not available")

print(f"\n✓ Executive summary completed")

In [ ]:
# Cell [11] - Statistical Hypothesis Testing for >30% Improvement in Time-to-Market
# Purpose: Perform formal statistical testing to validate hypothesis about hallucination-reducing techniques
# Dependencies: performance_df, effort_impact_df, scipy.stats for statistical tests
# Breadcrumbs: Setup -> Analysis -> Executive Summary -> Statistical Hypothesis Testing

from scipy.stats import ttest_ind, mannwhitneyu, chi2_contingency
from scipy import stats
import numpy as np

def perform_hypothesis_testing():
    """
    Perform formal statistical hypothesis testing for the >30% improvement claim
    
    Returns:
        dict: Statistical test results including p-values and confidence intervals
    """
    
    print("STATISTICAL HYPOTHESIS TESTING")
    print("=" * 60)
    print("HYPOTHESIS: Implementing hallucination-reducing techniques in LLMs")
    print("significantly improve (>30%) time to market in new product development")
    print("\nOPERATIONAL DEFINITIONS:")
    print("- Hallucination-reducing techniques: Multi-stage refinement (actor→judge→meta-judge)")
    print("- Time-to-market improvement: Measured via estimation accuracy reducing project delays")
    print("- Significance threshold: >30% improvement with p < 0.05")
    
    if performance_df.empty or effort_impact_df.empty:
        print("\nWarning: Insufficient data for statistical testing")
        return {}
    
    # Define treatment vs control groups based on model characteristics
    # Assumption: Models with judge scores > 3.5 represent "hallucination-reducing" techniques
    treatment_threshold = 3.5
    
    # Categorize models
    treatment_models = performance_df[performance_df['Avg_Judge_Score'] > treatment_threshold]
    control_models = performance_df[performance_df['Avg_Judge_Score'] <= treatment_threshold]
    
    print(f"\nGROUP DEFINITIONS:")
    print(f"Treatment group (Judge Score > {treatment_threshold}): {len(treatment_models)} models")
    print(f"Control group (Judge Score ≤ {treatment_threshold}): {len(control_models)} models")
    
    if len(treatment_models) == 0 or len(control_models) == 0:
        print("\nWarning: Insufficient models in treatment or control groups for comparison")
        print("Adjusting criteria...")
        
        # Alternative grouping: Top 50% vs bottom 50% by judge score
        median_judge_score = performance_df['Avg_Judge_Score'].median()
        treatment_models = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
        control_models = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
        
        print(f"Alternative grouping by median judge score ({median_judge_score:.2f}):")
        print(f"Treatment group: {len(treatment_models)} models")
        print(f"Control group: {len(control_models)} models")
    
    # Primary outcome: Estimation accuracy (lower error = better time-to-market)
    treatment_errors = treatment_models['Error_Pct'].values
    control_errors = control_models['Error_Pct'].values
    
    # Secondary outcomes: Success rate, effort estimation accuracy
    treatment_success = treatment_models['Success_Rate'].values
    control_success = control_models['Success_Rate'].values
    
    print(f"\nDESCRIPTIVE STATISTICS:")
    print(f"Treatment Group (n={len(treatment_errors)}):")
    print(f"  Mean error: {np.mean(treatment_errors):.2f}% (±{np.std(treatment_errors):.2f})")
    print(f"  Mean success rate: {np.mean(treatment_success):.2%} (±{np.std(treatment_success):.2%})")
    
    print(f"\nControl Group (n={len(control_errors)}):")
    print(f"  Mean error: {np.mean(control_errors):.2f}% (±{np.std(control_errors):.2f})")
    print(f"  Mean success rate: {np.mean(control_success):.2%} (±{np.std(control_success):.2%})")
    
    # Calculate improvement percentages
    error_improvement = (np.mean(control_errors) - np.mean(treatment_errors)) / np.mean(control_errors) * 100
    success_improvement = (np.mean(treatment_success) - np.mean(control_success)) / np.mean(control_success) * 100
    
    print(f"\nIMPROVEMENT ANALYSIS:")
    print(f"  Error reduction: {error_improvement:+.1f}%")
    print(f"  Success rate improvement: {success_improvement:+.1f}%")
    print(f"  Meets >30% threshold: {'YES' if abs(error_improvement) > 30 or success_improvement > 30 else 'NO'}")
    
    # Statistical tests
    test_results = {}
    
    # 1. T-test for estimation errors (assuming normal distribution)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        t_stat, t_pvalue = ttest_ind(treatment_errors, control_errors)
        
        # Calculate confidence interval for difference
        pooled_se = np.sqrt(np.var(treatment_errors)/len(treatment_errors) + 
                           np.var(control_errors)/len(control_errors))
        mean_diff = np.mean(treatment_errors) - np.mean(control_errors)
        margin_error = 1.96 * pooled_se  # 95% CI
        ci_lower = mean_diff - margin_error
        ci_upper = mean_diff + margin_error
        
        test_results['t_test'] = {
            'statistic': t_stat,
            'p_value': t_pvalue,
            'mean_difference': mean_diff,
            'ci_95': (ci_lower, ci_upper),
            'significant': t_pvalue < 0.05
        }
    
    # 2. Mann-Whitney U test (non-parametric alternative)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        u_stat, u_pvalue = mannwhitneyu(treatment_errors, control_errors, alternative='two-sided')
        
        test_results['mann_whitney'] = {
            'statistic': u_stat,
            'p_value': u_pvalue,
            'significant': u_pvalue < 0.05
        }
    
    # 3. Effect size (Cohen's d)
    if len(treatment_errors) > 1 and len(control_errors) > 1:
        pooled_std = np.sqrt(((len(treatment_errors)-1)*np.var(treatment_errors) + 
                             (len(control_errors)-1)*np.var(control_errors)) / 
                            (len(treatment_errors) + len(control_errors) - 2))
        cohens_d = (np.mean(treatment_errors) - np.mean(control_errors)) / pooled_std
        
        test_results['effect_size'] = {
            'cohens_d': cohens_d,
            'interpretation': 'small' if abs(cohens_d) < 0.5 else 'medium' if abs(cohens_d) < 0.8 else 'large'
        }
    
    # Display statistical test results
    print(f"\nSTATISTICAL TEST RESULTS:")
    print("=" * 40)
    
    if 't_test' in test_results:
        t_result = test_results['t_test']
        print(f"\n1. Independent Samples T-Test:")
        print(f"   H₀: No difference in estimation errors between groups")
        print(f"   H₁: Significant difference exists")
        print(f"   t-statistic: {t_result['statistic']:.3f}")
        print(f"   p-value: {t_result['p_value']:.4f}")
        print(f"   Mean difference: {t_result['mean_difference']:.2f}% error")
        print(f"   95% CI: ({t_result['ci_95'][0]:.2f}, {t_result['ci_95'][1]:.2f})")
        print(f"   Significant: {'YES' if t_result['significant'] else 'NO'} (α = 0.05)")
    
    if 'mann_whitney' in test_results:
        u_result = test_results['mann_whitney']
        print(f"\n2. Mann-Whitney U Test (Non-parametric):")
        print(f"   U-statistic: {u_result['statistic']:.3f}")
        print(f"   p-value: {u_result['p_value']:.4f}")
        print(f"   Significant: {'YES' if u_result['significant'] else 'NO'} (α = 0.05)")
    
    if 'effect_size' in test_results:
        effect = test_results['effect_size']
        print(f"\n3. Effect Size Analysis:")
        print(f"   Cohen's d: {effect['cohens_d']:.3f}")
        print(f"   Interpretation: {effect['interpretation']} effect")
    
    # Power analysis (post-hoc)
    if 'effect_size' in test_results and len(treatment_errors) > 1:
        from scipy.stats import norm
        alpha = 0.05
        n1, n2 = len(treatment_errors), len(control_errors)
        effect_size = abs(test_results['effect_size']['cohens_d'])
        
        # Simplified power calculation
        se = np.sqrt(1/n1 + 1/n2)
        critical_t = norm.ppf(1 - alpha/2)
        power = 1 - norm.cdf(critical_t - effect_size/se) + norm.cdf(-critical_t - effect_size/se)
        
        print(f"\n4. Statistical Power Analysis:")
        print(f"   Observed power: {power:.3f}")
        print(f"   Sample size (treatment): {n1}")
        print(f"   Sample size (control): {n2}")
        print(f"   Power interpretation: {'Adequate' if power > 0.8 else 'Inadequate'} (target: 0.8)")
    
    # Conclusion for hypothesis
    print(f"\nHYPOTHESIS TESTING CONCLUSION:")
    print("=" * 50)
    
    significant_improvement = (abs(error_improvement) > 30 or success_improvement > 30)
    statistically_significant = (test_results.get('t_test', {}).get('significant', False) or 
                               test_results.get('mann_whitney', {}).get('significant', False))
    
    print(f"\n✓ Magnitude Test: {'PASS' if significant_improvement else 'FAIL'}")
    print(f"  - Required: >30% improvement")
    print(f"  - Observed: {max(abs(error_improvement), success_improvement):.1f}% improvement")
    
    print(f"\n✓ Statistical Significance: {'PASS' if statistically_significant else 'FAIL'}")
    print(f"  - Required: p < 0.05")
    print(f"  - Observed: p = {test_results.get('t_test', {}).get('p_value', 'N/A')}")
    
    hypothesis_supported = significant_improvement and statistically_significant
    
    print(f"\n🎯 FINAL VERDICT: {'HYPOTHESIS SUPPORTED' if hypothesis_supported else 'HYPOTHESIS NOT SUPPORTED'}")
    
    if not hypothesis_supported:
        print(f"\nRECOMMENDATIONS FOR FUTURE RESEARCH:")
        print(f"  1. Increase sample size (current: {len(performance_df)} models)")
        print(f"  2. Define clearer treatment/control groups")
        print(f"  3. Collect direct time-to-market measurements")
        print(f"  4. Implement randomized controlled trial design")
        print(f"  5. Establish baseline measurements before intervention")
    
    return test_results

# Execute statistical hypothesis testing
if 'performance_df' in globals() and not performance_df.empty:
    statistical_results = perform_hypothesis_testing()
else:
    print("Cannot perform hypothesis testing - performance data not available")
    statistical_results = {}


In [ ]:
# Cell [11A] - Advanced Permutation Testing for Hallucination-Reducing Techniques
# Purpose: Implement robust permutation tests that don't rely on distributional assumptions
# Dependencies: performance_df from Cell 5, STATISTICAL_CONFIG from Cell 0, scipy.stats permutation_test
# Breadcrumbs: Setup -> Analysis -> Statistical Testing -> Advanced Permutation Testing

def perform_advanced_permutation_testing():
    """
    Perform comprehensive permutation testing for the >30% improvement hypothesis
    
    Returns:
        dict: Permutation test results with exact p-values and effect sizes
    """
    
    print("ADVANCED PERMUTATION TESTING")
    print("=" * 60)
    print("HYPOTHESIS: Hallucination-reducing techniques improve time-to-market by >30%")
    print("METHOD: Distribution-free permutation tests with exact p-values")
    
    if performance_df.empty:
        print("\nWarning: Insufficient data for permutation testing")
        return {}
    
    # Define treatment vs control groups (same logic as Cell 11 but enhanced)
    median_judge_score = performance_df['Avg_Judge_Score'].median()
    treatment_models = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
    control_models = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
    
    print(f"\nGROUP DEFINITIONS:")
    print(f"Treatment group (Judge Score > {median_judge_score:.2f}): {len(treatment_models)} models")
    print(f"Control group (Judge Score ≤ {median_judge_score:.2f}): {len(control_models)} models")
    
    if len(treatment_models) == 0 or len(control_models) == 0:
        print("Warning: Cannot perform permutation tests - insufficient group sizes")
        return {}
    
    # Extract data arrays for testing
    treatment_errors = treatment_models['Error_Pct'].values
    control_errors = control_models['Error_Pct'].values
    treatment_success = treatment_models['Success_Rate'].values
    control_success = control_models['Success_Rate'].values
    
    permutation_results = {}
    n_resamples = CONFIG.get('PERMUTATION_SAMPLES', 10000)
    
    print(f"\nPERMUTATION TEST CONFIGURATION:")
    print(f"  Number of resamples: {n_resamples:,}")
    print(f"  Random seed: 42 (for reproducibility)")
    print(f"  Test type: Two-sided")
    
    # 1. Permutation test for estimation error reduction
    print(f"\n" + "="*50)
    print("1. PERMUTATION TEST: ESTIMATION ERROR REDUCTION")
    print("="*50)
    
    print(f"H₀: No difference in estimation errors between groups")
    print(f"H₁: Treatment group has different (lower) estimation errors")
    
    def error_difference_statistic(treatment, control):
        """Test statistic: difference in means (control - treatment)"""
        return np.mean(control) - np.mean(treatment)
    
    try:
        # Perform permutation test for error reduction
        error_perm_result = permutation_test(
            (treatment_errors, control_errors),
            error_difference_statistic,
            n_resamples=n_resamples,
            alternative='greater',  # We expect treatment to have lower errors
            random_state=42
        )
        
        observed_error_reduction = error_difference_statistic(treatment_errors, control_errors)
        error_improvement_pct = (observed_error_reduction / np.mean(control_errors)) * 100
        
        print(f"\nResults:")
        print(f"  Observed error reduction: {observed_error_reduction:.2f} percentage points")
        print(f"  Improvement percentage: {error_improvement_pct:.1f}%")
        print(f"  Permutation p-value: {error_perm_result.pvalue:.6f}")
        print(f"  Significant: {'YES' if error_perm_result.pvalue < CONFIG['ALPHA_LEVEL'] else 'NO'}")
        print(f"  Meets >30% threshold: {'YES' if abs(error_improvement_pct) > 30 else 'NO'}")
        
        permutation_results['error_reduction'] = {
            'observed_statistic': observed_error_reduction,
            'improvement_pct': error_improvement_pct,
            'p_value': error_perm_result.pvalue,
            'significant': error_perm_result.pvalue < CONFIG['ALPHA_LEVEL'],
            'meets_threshold': abs(error_improvement_pct) > 30,
            'null_distribution': error_perm_result.null_distribution
        }
        
    except Exception as e:
        print(f"Error in error reduction permutation test: {e}")
        permutation_results['error_reduction'] = {}
    
    # 2. Permutation test for success rate improvement
    print(f"\n" + "="*50)
    print("2. PERMUTATION TEST: SUCCESS RATE IMPROVEMENT")
    print("="*50)
    
    print(f"H₀: No difference in success rates between groups")
    print(f"H₁: Treatment group has higher success rates")
    
    def success_difference_statistic(treatment, control):
        """Test statistic: difference in means (treatment - control)"""
        return np.mean(treatment) - np.mean(control)
    
    try:
        # Perform permutation test for success rate improvement
        success_perm_result = permutation_test(
            (treatment_success, control_success),
            success_difference_statistic,
            n_resamples=n_resamples,
            alternative='greater',  # We expect treatment to have higher success rates
            random_state=42
        )
        
        observed_success_improvement = success_difference_statistic(treatment_success, control_success)
        success_improvement_pct = (observed_success_improvement / np.mean(control_success)) * 100
        
        print(f"\nResults:")
        print(f"  Observed success improvement: {observed_success_improvement:.4f}")
        print(f"  Improvement percentage: {success_improvement_pct:.1f}%")
        print(f"  Permutation p-value: {success_perm_result.pvalue:.6f}")
        print(f"  Significant: {'YES' if success_perm_result.pvalue < CONFIG['ALPHA_LEVEL'] else 'NO'}")
        print(f"  Meets >30% threshold: {'YES' if success_improvement_pct > 30 else 'NO'}")
        
        permutation_results['success_improvement'] = {
            'observed_statistic': observed_success_improvement,
            'improvement_pct': success_improvement_pct,
            'p_value': success_perm_result.pvalue,
            'significant': success_perm_result.pvalue < CONFIG['ALPHA_LEVEL'],
            'meets_threshold': success_improvement_pct > 30,
            'null_distribution': success_perm_result.null_distribution
        }
        
    except Exception as e:
        print(f"Error in success rate permutation test: {e}")
        permutation_results['success_improvement'] = {}
    
    # 3. Combined permutation test for overall improvement
    print(f"\n" + "="*50)
    print("3. COMBINED PERMUTATION TEST: OVERALL IMPROVEMENT")
    print("="*50)
    
    print(f"H₀: No overall improvement in treatment group")
    print(f"H₁: Treatment group shows overall improvement (error reduction OR success improvement)")
    
    def combined_improvement_statistic(treatment_err, control_err, treatment_succ, control_succ):
        """Combined test statistic: max of standardized improvements"""
        error_improvement = (np.mean(control_err) - np.mean(treatment_err)) / np.std(control_err)
        success_improvement = (np.mean(treatment_succ) - np.mean(control_succ)) / np.std(control_succ)
        return max(error_improvement, success_improvement)
    
    try:
        # Custom permutation test for combined statistic
        def combined_statistic_wrapper(combined_data):
            """Wrapper for combined data permutation"""
            n_treatment = len(treatment_errors)
            treatment_err_perm = combined_data[0][:n_treatment]
            control_err_perm = combined_data[0][n_treatment:]
            treatment_succ_perm = combined_data[1][:n_treatment]
            control_succ_perm = combined_data[1][n_treatment:]
            return combined_improvement_statistic(treatment_err_perm, control_err_perm, 
                                                treatment_succ_perm, control_succ_perm)
        
        # Combine data for permutation
        combined_errors = np.concatenate([treatment_errors, control_errors])
        combined_success = np.concatenate([treatment_success, control_success])
        
        combined_perm_result = permutation_test(
            (combined_errors, combined_success),
            combined_statistic_wrapper,
            n_resamples=n_resamples,
            alternative='greater',
            random_state=42
        )
        
        observed_combined = combined_improvement_statistic(treatment_errors, control_errors, 
                                                         treatment_success, control_success)
        
        print(f"\nResults:")
        print(f"  Combined improvement statistic: {observed_combined:.3f}")
        print(f"  Permutation p-value: {combined_perm_result.pvalue:.6f}")
        print(f"  Significant: {'YES' if combined_perm_result.pvalue < CONFIG['ALPHA_LEVEL'] else 'NO'}")
        
        permutation_results['combined_improvement'] = {
            'observed_statistic': observed_combined,
            'p_value': combined_perm_result.pvalue,
            'significant': combined_perm_result.pvalue < CONFIG['ALPHA_LEVEL'],
            'null_distribution': combined_perm_result.null_distribution
        }
        
    except Exception as e:
        print(f"Error in combined permutation test: {e}")
        permutation_results['combined_improvement'] = {}
    
    # 4. Permutation-based confidence intervals
    print(f"\n" + "="*50)
    print("4. PERMUTATION-BASED CONFIDENCE INTERVALS")
    print("="*50)
    
    try:
        # Bootstrap-style permutation for confidence intervals
        def permutation_ci(data1, data2, statistic_func, confidence_level=0.95, n_resamples=n_resamples):
            """Calculate permutation-based confidence interval"""
            combined_data = np.concatenate([data1, data2])
            n1 = len(data1)
            
            # Generate permutation statistics
            perm_statistics = []
            np.random.seed(42)
            
            for _ in range(n_resamples):
                perm_data = np.random.permutation(combined_data)
                perm1 = perm_data[:n1]
                perm2 = perm_data[n1:]
                perm_stat = statistic_func(perm1, perm2)
                perm_statistics.append(perm_stat)
            
            # Calculate confidence interval
            alpha = 1 - confidence_level
            lower_percentile = (alpha / 2) * 100
            upper_percentile = (1 - alpha / 2) * 100
            
            ci_lower = np.percentile(perm_statistics, lower_percentile)
            ci_upper = np.percentile(perm_statistics, upper_percentile)
            
            return ci_lower, ci_upper, np.array(perm_statistics)
        
        # CI for error reduction
        error_ci_lower, error_ci_upper, error_perm_dist = permutation_ci(
            treatment_errors, control_errors, error_difference_statistic
        )
        
        # CI for success improvement
        success_ci_lower, success_ci_upper, success_perm_dist = permutation_ci(
            treatment_success, control_success, success_difference_statistic
        )
        
        print(f"95% Confidence Intervals (Permutation-based):")
        print(f"  Error reduction: [{error_ci_lower:.2f}, {error_ci_upper:.2f}] percentage points")
        print(f"  Success improvement: [{success_ci_lower:.4f}, {success_ci_upper:.4f}]")
        
        # Check if CI excludes zero (indicates significance)
        error_ci_significant = error_ci_lower > 0 or error_ci_upper < 0
        success_ci_significant = success_ci_lower > 0 or success_ci_upper < 0
        
        print(f"\nConfidence Interval Significance:")
        print(f"  Error reduction CI excludes 0: {'YES' if error_ci_significant else 'NO'}")
        print(f"  Success improvement CI excludes 0: {'YES' if success_ci_significant else 'NO'}")
        
        permutation_results['confidence_intervals'] = {
            'error_reduction_ci': (error_ci_lower, error_ci_upper),
            'success_improvement_ci': (success_ci_lower, success_ci_upper),
            'error_ci_significant': error_ci_significant,
            'success_ci_significant': success_ci_significant
        }
        
    except Exception as e:
        print(f"Error calculating permutation confidence intervals: {e}")
        permutation_results['confidence_intervals'] = {}
    
    # 5. Multiple comparisons adjustment
    print(f"\n" + "="*50)
    print("5. MULTIPLE COMPARISONS ADJUSTMENT")
    print("="*50)
    
    try:
        from statsmodels.stats.multitest import multipletests
        
        # Collect all p-values for adjustment
        p_values = []
        test_names = []
        
        if 'error_reduction' in permutation_results and permutation_results['error_reduction']:
            p_values.append(permutation_results['error_reduction']['p_value'])
            test_names.append('Error Reduction')
        
        if 'success_improvement' in permutation_results and permutation_results['success_improvement']:
            p_values.append(permutation_results['success_improvement']['p_value'])
            test_names.append('Success Improvement')
        
        if 'combined_improvement' in permutation_results and permutation_results['combined_improvement']:
            p_values.append(permutation_results['combined_improvement']['p_value'])
            test_names.append('Combined Improvement')
        
        if p_values:
            # Apply Bonferroni correction
            bonferroni_rejected, bonferroni_pvals, _, _ = multipletests(p_values, method='bonferroni')
            
            # Apply Benjamini-Hochberg (FDR) correction
            fdr_rejected, fdr_pvals, _, _ = multipletests(p_values, method='fdr_bh')
            
            print(f"Multiple Comparisons Results:")
            print(f"{'Test':<20} {'Raw p-value':<12} {'Bonferroni':<12} {'FDR (B-H)':<12}")
            print("-" * 60)
            
            for i, (test_name, raw_p) in enumerate(zip(test_names, p_values)):
                print(f"{test_name:<20} {raw_p:<12.6f} {bonferroni_pvals[i]:<12.6f} {fdr_pvals[i]:<12.6f}")
            
            permutation_results['multiple_comparisons'] = {
                'raw_p_values': p_values,
                'test_names': test_names,
                'bonferroni_p_values': bonferroni_pvals.tolist(),
                'bonferroni_rejected': bonferroni_rejected.tolist(),
                'fdr_p_values': fdr_pvals.tolist(),
                'fdr_rejected': fdr_rejected.tolist()
            }
    
    except Exception as e:
        print(f"Error in multiple comparisons adjustment: {e}")
        permutation_results['multiple_comparisons'] = {}
    
    # Summary of permutation testing results
    print(f"\n" + "="*60)
    print("PERMUTATION TESTING SUMMARY")
    print("="*60)
    
    significant_tests = 0
    total_tests = 0
    meets_threshold_tests = 0
    
    for test_name, results in permutation_results.items():
        if test_name == 'multiple_comparisons' or test_name == 'confidence_intervals':
            continue
        if results and 'significant' in results:
            total_tests += 1
            if results['significant']:
                significant_tests += 1
            if results.get('meets_threshold', False):
                meets_threshold_tests += 1
    
    print(f"\nOverall Results:")
    print(f"  Total permutation tests: {total_tests}")
    print(f"  Statistically significant: {significant_tests}/{total_tests}")
    print(f"  Meet >30% threshold: {meets_threshold_tests}/{total_tests}")
    
    # Final verdict for permutation testing
    any_significant = significant_tests > 0
    any_threshold = meets_threshold_tests > 0
    
    print(f"\n🎯 PERMUTATION TESTING VERDICT:")
    print(f"  Statistical Evidence: {'Found' if any_significant else 'Not Found'}")
    print(f"  Practical Significance: {'Found' if any_threshold else 'Not Found'}")
    print(f"  Overall Support: {'STRONG' if any_significant and any_threshold else 'MODERATE' if any_significant or any_threshold else 'WEAK'}")
    
    print(f"\nADVANTAGES OF PERMUTATION TESTING:")
    print(f"  ✓ No distributional assumptions required")
    print(f"  ✓ Exact p-values (not asymptotic)")
    print(f"  ✓ Robust to outliers")
    print(f"  ✓ Suitable for small sample sizes")
    print(f"  ✓ Multiple comparison corrections applied")
    
    return permutation_results

# Execute advanced permutation testing
if 'performance_df' in globals() and not performance_df.empty and STATISTICAL_CONFIG.get('permutation_tests', False):
    permutation_results = perform_advanced_permutation_testing()
    print(f"\n✓ Advanced permutation testing completed successfully")
else:
    print("Cannot perform advanced permutation testing:")
    if 'performance_df' not in globals() or performance_df.empty:
        print("  - Performance data not available")
    if not STATISTICAL_CONFIG.get('permutation_tests', False):
        print("  - Permutation testing functionality not available")
        print("  - Install with: pip install 'praxis-requirements-analyzer[dev]'")
    
    permutation_results = {}

In [ ]:
# Cell [11B] - Expanded Bootstrap Hypothesis Testing
# Purpose: Bootstrap-based hypothesis testing for >30% improvement with confidence intervals
# Dependencies: performance_df, effort_impact_df, bootstrap functionality from Cell 0
# Breadcrumbs: Setup -> Analysis -> Permutation Testing -> Bootstrap Hypothesis Testing

def perform_bootstrap_hypothesis_testing():
    """
    Perform comprehensive bootstrap hypothesis testing for the >30% improvement claim
    
    Returns:
        dict: Bootstrap test results with confidence intervals and bias corrections
    """
    
    print("EXPANDED BOOTSTRAP HYPOTHESIS TESTING")
    print("=" * 60)
    print("HYPOTHESIS: Hallucination-reducing techniques improve time-to-market by >30%")
    print("METHOD: Bootstrap resampling with bias-corrected confidence intervals")
    
    if performance_df.empty:
        print("\nWarning: Insufficient data for bootstrap testing")
        return {}
    
    # Define treatment vs control groups (consistent with Cell 11A)
    median_judge_score = performance_df['Avg_Judge_Score'].median()
    treatment_models = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
    control_models = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
    
    print(f"\nGROUP DEFINITIONS:")
    print(f"Treatment group (Judge Score > {median_judge_score:.2f}): {len(treatment_models)} models")
    print(f"Control group (Judge Score ≤ {median_judge_score:.2f}): {len(control_models)} models")
    
    if len(treatment_models) == 0 or len(control_models) == 0:
        print("Warning: Cannot perform bootstrap tests - insufficient group sizes")
        return {}
    
    # Extract data arrays for testing
    treatment_errors = treatment_models['Error_Pct'].values
    control_errors = control_models['Error_Pct'].values
    treatment_success = treatment_models['Success_Rate'].values
    control_success = control_models['Success_Rate'].values
    
    bootstrap_results = {}
    n_bootstrap = CONFIG.get('BOOTSTRAP_SAMPLES', 10000)
    
    print(f"\nBOOTSTRAP TEST CONFIGURATION:")
    print(f"  Number of bootstrap samples: {n_bootstrap:,}")
    print(f"  Random seed: 42 (for reproducibility)")
    print(f"  Confidence level: 95%")
    print(f"  Bias correction: BCa (Bias-Corrected and accelerated)")
    
    # 1. Bootstrap test for error reduction
    print(f"\n" + "="*50)
    print("1. BOOTSTRAP TEST: ESTIMATION ERROR REDUCTION")
    print("="*50)
    
    print(f"H₀: No difference in estimation errors between groups")
    print(f"H₁: Treatment group has lower estimation errors (>30% improvement)")
    
    try:
        def error_improvement_statistic(control_sample, treatment_sample):
            """Calculate improvement percentage: (control - treatment) / control * 100"""
            control_mean = np.mean(control_sample)
            treatment_mean = np.mean(treatment_sample)
            if control_mean == 0:
                return 0
            return ((control_mean - treatment_mean) / control_mean) * 100
        
        # Bootstrap for error improvement
        def bootstrap_error_improvement(n_samples=n_bootstrap):
            improvements = []
            np.random.seed(42)
            
            for _ in range(n_samples):
                # Resample with replacement from each group
                control_sample = np.random.choice(control_errors, len(control_errors), replace=True)
                treatment_sample = np.random.choice(treatment_errors, len(treatment_errors), replace=True)
                
                improvement = error_improvement_statistic(control_sample, treatment_sample)
                improvements.append(improvement)
            
            return np.array(improvements)
        
        # Generate bootstrap distribution
        error_bootstrap_dist = bootstrap_error_improvement()
        observed_error_improvement = error_improvement_statistic(control_errors, treatment_errors)
        
        # Calculate confidence intervals
        error_ci_lower = np.percentile(error_bootstrap_dist, 2.5)
        error_ci_upper = np.percentile(error_bootstrap_dist, 97.5)
        
        # Bootstrap hypothesis test: H₀: improvement ≤ 0
        # Count how many bootstrap samples show improvement ≤ 0
        null_violations = np.sum(error_bootstrap_dist <= 0)
        error_bootstrap_p_value = null_violations / len(error_bootstrap_dist)
        
        # Test for >30% improvement specifically
        improvement_30_violations = np.sum(error_bootstrap_dist < 30)
        improvement_30_p_value = improvement_30_violations / len(error_bootstrap_dist)
        
        print(f"\nResults:")
        print(f"  Observed error improvement: {observed_error_improvement:.1f}%")
        print(f"  Bootstrap mean improvement: {np.mean(error_bootstrap_dist):.1f}%")
        print(f"  Bootstrap std improvement: {np.std(error_bootstrap_dist):.1f}%")
        print(f"  95% Bootstrap CI: [{error_ci_lower:.1f}%, {error_ci_upper:.1f}%]")
        print(f"  P(improvement ≤ 0): {error_bootstrap_p_value:.4f}")
        print(f"  P(improvement < 30%): {improvement_30_p_value:.4f}")
        print(f"  Significant improvement: {'YES' if error_bootstrap_p_value < CONFIG['ALPHA_LEVEL'] else 'NO'}")
        print(f"  Meets >30% threshold: {'YES' if observed_error_improvement > 30 else 'NO'}")
        print(f"  30% threshold confidence: {1 - improvement_30_p_value:.3f}")
        
        bootstrap_results['error_reduction'] = {
            'observed_improvement': observed_error_improvement,
            'bootstrap_mean': np.mean(error_bootstrap_dist),
            'bootstrap_std': np.std(error_bootstrap_dist),
            'confidence_interval': (error_ci_lower, error_ci_upper),
            'p_value_improvement': error_bootstrap_p_value,
            'p_value_30_percent': improvement_30_p_value,
            'significant': error_bootstrap_p_value < CONFIG['ALPHA_LEVEL'],
            'meets_threshold': observed_error_improvement > 30,
            'threshold_confidence': 1 - improvement_30_p_value,
            'bootstrap_distribution': error_bootstrap_dist
        }
        
    except Exception as e:
        print(f"Error in bootstrap error reduction test: {e}")
        bootstrap_results['error_reduction'] = {}
    
    # 2. Bootstrap test for success rate improvement
    print(f"\n" + "="*50)
    print("2. BOOTSTRAP TEST: SUCCESS RATE IMPROVEMENT")
    print("="*50)
    
    print(f"H₀: No difference in success rates between groups")
    print(f"H₁: Treatment group has higher success rates (>30% improvement)")
    
    try:
        def success_improvement_statistic(control_sample, treatment_sample):
            """Calculate improvement percentage: (treatment - control) / control * 100"""
            control_mean = np.mean(control_sample)
            treatment_mean = np.mean(treatment_sample)
            if control_mean == 0:
                return 0
            return ((treatment_mean - control_mean) / control_mean) * 100
        
        # Bootstrap for success rate improvement
        def bootstrap_success_improvement(n_samples=n_bootstrap):
            improvements = []
            np.random.seed(42)
            
            for _ in range(n_samples):
                # Resample with replacement from each group
                control_sample = np.random.choice(control_success, len(control_success), replace=True)
                treatment_sample = np.random.choice(treatment_success, len(treatment_success), replace=True)
                
                improvement = success_improvement_statistic(control_sample, treatment_sample)
                improvements.append(improvement)
            
            return np.array(improvements)
        
        # Generate bootstrap distribution
        success_bootstrap_dist = bootstrap_success_improvement()
        observed_success_improvement = success_improvement_statistic(control_success, treatment_success)
        
        # Calculate confidence intervals
        success_ci_lower = np.percentile(success_bootstrap_dist, 2.5)
        success_ci_upper = np.percentile(success_bootstrap_dist, 97.5)
        
        # Bootstrap hypothesis test: H₀: improvement ≤ 0
        null_violations = np.sum(success_bootstrap_dist <= 0)
        success_bootstrap_p_value = null_violations / len(success_bootstrap_dist)
        
        # Test for >30% improvement specifically
        improvement_30_violations = np.sum(success_bootstrap_dist < 30)
        success_30_p_value = improvement_30_violations / len(success_bootstrap_dist)
        
        print(f"\nResults:")
        print(f"  Observed success improvement: {observed_success_improvement:.1f}%")
        print(f"  Bootstrap mean improvement: {np.mean(success_bootstrap_dist):.1f}%")
        print(f"  Bootstrap std improvement: {np.std(success_bootstrap_dist):.1f}%")
        print(f"  95% Bootstrap CI: [{success_ci_lower:.1f}%, {success_ci_upper:.1f}%]")
        print(f"  P(improvement ≤ 0): {success_bootstrap_p_value:.4f}")
        print(f"  P(improvement < 30%): {success_30_p_value:.4f}")
        print(f"  Significant improvement: {'YES' if success_bootstrap_p_value < CONFIG['ALPHA_LEVEL'] else 'NO'}")
        print(f"  Meets >30% threshold: {'YES' if observed_success_improvement > 30 else 'NO'}")
        print(f"  30% threshold confidence: {1 - success_30_p_value:.3f}")
        
        bootstrap_results['success_improvement'] = {
            'observed_improvement': observed_success_improvement,
            'bootstrap_mean': np.mean(success_bootstrap_dist),
            'bootstrap_std': np.std(success_bootstrap_dist),
            'confidence_interval': (success_ci_lower, success_ci_upper),
            'p_value_improvement': success_bootstrap_p_value,
            'p_value_30_percent': success_30_p_value,
            'significant': success_bootstrap_p_value < CONFIG['ALPHA_LEVEL'],
            'meets_threshold': observed_success_improvement > 30,
            'threshold_confidence': 1 - success_30_p_value,
            'bootstrap_distribution': success_bootstrap_dist
        }
        
    except Exception as e:
        print(f"Error in bootstrap success rate test: {e}")
        bootstrap_results['success_improvement'] = {}
    
    # 3. Bias-Corrected and Accelerated (BCa) Bootstrap Intervals
    print(f"\n" + "="*50)
    print("3. BIAS-CORRECTED AND ACCELERATED (BCa) INTERVALS")
    print("="*50)
    
    try:
        def calculate_bca_interval(data1, data2, statistic_func, alpha=0.05, n_bootstrap=n_bootstrap):
            """
            Calculate BCa (Bias-Corrected and accelerated) bootstrap confidence interval
            """
            # Original statistic
            original_stat = statistic_func(data1, data2)
            
            # Bootstrap statistics
            bootstrap_stats = []
            np.random.seed(42)
            
            for _ in range(n_bootstrap):
                sample1 = np.random.choice(data1, len(data1), replace=True)
                sample2 = np.random.choice(data2, len(data2), replace=True)
                bootstrap_stats.append(statistic_func(sample1, sample2))
            
            bootstrap_stats = np.array(bootstrap_stats)
            
            # Bias correction
            num_less = np.sum(bootstrap_stats < original_stat)
            bias_correction = stats.norm.ppf(num_less / n_bootstrap)
            
            # Acceleration (jackknife)
            n1, n2 = len(data1), len(data2)
            jackknife_stats = []
            
            # Jackknife for data1
            for i in range(n1):
                jack_data1 = np.delete(data1, i)
                jackknife_stats.append(statistic_func(jack_data1, data2))
            
            # Jackknife for data2
            for i in range(n2):
                jack_data2 = np.delete(data2, i)
                jackknife_stats.append(statistic_func(data1, jack_data2))
            
            jackknife_stats = np.array(jackknife_stats)
            jackknife_mean = np.mean(jackknife_stats)
            
            # Acceleration parameter
            numerator = np.sum((jackknife_mean - jackknife_stats)**3)
            denominator = 6 * (np.sum((jackknife_mean - jackknife_stats)**2))**1.5
            acceleration = numerator / denominator if denominator != 0 else 0
            
            # BCa percentiles
            z_alpha_2 = stats.norm.ppf(alpha/2)
            z_1_alpha_2 = stats.norm.ppf(1 - alpha/2)
            
            alpha_1 = stats.norm.cdf(bias_correction + (bias_correction + z_alpha_2)/(1 - acceleration*(bias_correction + z_alpha_2)))
            alpha_2 = stats.norm.cdf(bias_correction + (bias_correction + z_1_alpha_2)/(1 - acceleration*(bias_correction + z_1_alpha_2)))
            
            # Ensure percentiles are within valid range
            alpha_1 = max(0, min(1, alpha_1))
            alpha_2 = max(0, min(1, alpha_2))
            
            lower_percentile = alpha_1 * 100
            upper_percentile = alpha_2 * 100
            
            bca_lower = np.percentile(bootstrap_stats, lower_percentile)
            bca_upper = np.percentile(bootstrap_stats, upper_percentile)
            
            return bca_lower, bca_upper, bias_correction, acceleration
        
        # BCa intervals for error improvement
        def error_improvement_for_bca(control_sample, treatment_sample):
            return error_improvement_statistic(control_sample, treatment_sample)
        
        error_bca_lower, error_bca_upper, error_bias, error_accel = calculate_bca_interval(
            control_errors, treatment_errors, error_improvement_for_bca
        )
        
        # BCa intervals for success improvement
        def success_improvement_for_bca(control_sample, treatment_sample):
            return success_improvement_statistic(control_sample, treatment_sample)
        
        success_bca_lower, success_bca_upper, success_bias, success_accel = calculate_bca_interval(
            control_success, treatment_success, success_improvement_for_bca
        )
        
        print(f"BCa Confidence Intervals (95%):")
        print(f"  Error reduction:")
        print(f"    Standard CI: [{bootstrap_results['error_reduction']['confidence_interval'][0]:.1f}%, {bootstrap_results['error_reduction']['confidence_interval'][1]:.1f}%]")
        print(f"    BCa CI: [{error_bca_lower:.1f}%, {error_bca_upper:.1f}%]")
        print(f"    Bias correction: {error_bias:.3f}")
        print(f"    Acceleration: {error_accel:.3f}")
        
        print(f"  Success rate improvement:")
        print(f"    Standard CI: [{bootstrap_results['success_improvement']['confidence_interval'][0]:.1f}%, {bootstrap_results['success_improvement']['confidence_interval'][1]:.1f}%]")
        print(f"    BCa CI: [{success_bca_lower:.1f}%, {success_bca_upper:.1f}%]")
        print(f"    Bias correction: {success_bias:.3f}")
        print(f"    Acceleration: {success_accel:.3f}")
        
        bootstrap_results['bca_intervals'] = {
            'error_reduction_bca': (error_bca_lower, error_bca_upper),
            'success_improvement_bca': (success_bca_lower, success_bca_upper),
            'error_bias_correction': error_bias,
            'error_acceleration': error_accel,
            'success_bias_correction': success_bias,
            'success_acceleration': success_accel
        }
        
    except Exception as e:
        print(f"Error calculating BCa intervals: {e}")
        bootstrap_results['bca_intervals'] = {}
    
    # 4. Bootstrap Power Analysis
    print(f"\n" + "="*50)
    print("4. BOOTSTRAP POWER ANALYSIS")
    print("="*50)
    
    try:
        def bootstrap_power_analysis(effect_sizes, sample_sizes, alpha=0.05):
            """
            Estimate statistical power using bootstrap simulation
            """
            power_results = {}
            
            for n in sample_sizes:
                power_results[n] = {}
                for effect in effect_sizes:
                    # Simulate data with known effect size
                    sim_control = np.random.normal(50, 15, n)  # Mean error ~50%
                    sim_treatment = np.random.normal(50 * (1 - effect), 15, n)  # Reduced by effect
                    
                    # Bootstrap test
                    bootstrap_improvements = []
                    for _ in range(1000):  # Reduced for speed
                        boot_control = np.random.choice(sim_control, n, replace=True)
                        boot_treatment = np.random.choice(sim_treatment, n, replace=True)
                        improvement = error_improvement_statistic(boot_control, boot_treatment)
                        bootstrap_improvements.append(improvement)
                    
                    # Calculate power (proportion of significant results)
                    null_violations = np.sum(np.array(bootstrap_improvements) <= 0)
                    p_value = null_violations / len(bootstrap_improvements)
                    power = 1 - (p_value >= alpha)
                    
                    power_results[n][effect] = power
            
            return power_results
        
        effect_sizes = [0.1, 0.2, 0.3, 0.4, 0.5]  # 10%, 20%, 30%, 40%, 50% improvements
        sample_sizes = [5, 10, 15, 20] if len(performance_df) <= 20 else [10, 20, 30, 50]
        
        power_analysis = bootstrap_power_analysis(effect_sizes, sample_sizes)
        
        print(f"Bootstrap Power Analysis Results:")
        print(f"{'Sample Size':<12} {'10%':<8} {'20%':<8} {'30%':<8} {'40%':<8} {'50%':<8}")
        print("-" * 60)
        
        for n in sample_sizes:
            power_row = f"{n:<12}"
            for effect in effect_sizes:
                power_row += f"{power_analysis[n][effect]:.2f}    "
            print(power_row)
        
        bootstrap_results['power_analysis'] = power_analysis
        
    except Exception as e:
        print(f"Error in bootstrap power analysis: {e}")
        bootstrap_results['power_analysis'] = {}
    
    # 5. Bootstrap Hypothesis Testing Summary
    print(f"\n" + "="*60)
    print("BOOTSTRAP HYPOTHESIS TESTING SUMMARY")
    print("="*60)
    
    significant_tests = 0
    total_tests = 0
    meets_threshold_tests = 0
    high_confidence_tests = 0
    
    for test_name, results in bootstrap_results.items():
        if test_name in ['bca_intervals', 'power_analysis']:
            continue
        if results and 'significant' in results:
            total_tests += 1
            if results['significant']:
                significant_tests += 1
            if results.get('meets_threshold', False):
                meets_threshold_tests += 1
            if results.get('threshold_confidence', 0) > 0.8:
                high_confidence_tests += 1
    
    print(f"\nOverall Results:")
    print(f"  Total bootstrap tests: {total_tests}")
    print(f"  Statistically significant: {significant_tests}/{total_tests}")
    print(f"  Meet >30% threshold: {meets_threshold_tests}/{total_tests}")
    print(f"  High confidence (>80%) for 30% threshold: {high_confidence_tests}/{total_tests}")
    
    # Final verdict for bootstrap testing
    any_significant = significant_tests > 0
    any_threshold = meets_threshold_tests > 0
    high_confidence = high_confidence_tests > 0
    
    print(f"\n🎯 BOOTSTRAP TESTING VERDICT:")
    print(f"  Statistical Evidence: {'Found' if any_significant else 'Not Found'}")
    print(f"  Practical Significance: {'Found' if any_threshold else 'Not Found'}")
    print(f"  High Confidence in 30% Threshold: {'YES' if high_confidence else 'NO'}")
    print(f"  Overall Support: {'STRONG' if any_significant and any_threshold and high_confidence else 'MODERATE' if any_significant or any_threshold else 'WEAK'}")
    
    print(f"\nADVANTAGES OF BOOTSTRAP TESTING:")
    print(f"  ✓ Provides confidence intervals for complex statistics")
    print(f"  ✓ Accounts for bias in small samples (BCa intervals)")
    print(f"  ✓ Direct probability statements about thresholds")
    print(f"  ✓ Robust to non-normal distributions")
    print(f"  ✓ Intuitive interpretation of results")
    
    return bootstrap_results

# Execute bootstrap hypothesis testing
if 'performance_df' in globals() and not performance_df.empty and STATISTICAL_CONFIG.get('bootstrap_tests', False):
    bootstrap_results = perform_bootstrap_hypothesis_testing()
    print(f"\n✓ Bootstrap hypothesis testing completed successfully")
else:
    print("Cannot perform bootstrap hypothesis testing:")
    if 'performance_df' not in globals() or performance_df.empty:
        print("  - Performance data not available")
    if not STATISTICAL_CONFIG.get('bootstrap_tests', False):
        print("  - Bootstrap testing functionality not available")
        print("  - Install with: pip install 'praxis-requirements-analyzer[dev]'")
    
    bootstrap_results = {}

In [ ]:
# Cell [11C] - Bayesian Analysis for >30% Improvement Hypothesis
# Purpose: Bayesian hypothesis testing with posterior probability statements
# Dependencies: performance_df, pymc, arviz from Cell 0, BAYESIAN_CONFIG
# Breadcrumbs: Setup -> Analysis -> Bootstrap Testing -> Bayesian Analysis

def perform_bayesian_analysis():
    """
    Perform comprehensive Bayesian analysis for the >30% improvement hypothesis
    
    Returns:
        dict: Bayesian analysis results with posterior probabilities and credible intervals
    """
    
    print("BAYESIAN ANALYSIS FOR >30% IMPROVEMENT HYPOTHESIS")
    print("=" * 60)
    print("HYPOTHESIS: Hallucination-reducing techniques improve time-to-market by >30%")
    print("METHOD: Bayesian inference with posterior probability statements")
    
    if performance_df.empty or not STATISTICAL_CONFIG.get('bayesian_analysis', False):
        print("\nWarning: Insufficient data or Bayesian functionality not available")
        return {}
    
    # Define treatment vs control groups (consistent with Cells 11A and 11B)
    median_judge_score = performance_df['Avg_Judge_Score'].median()
    treatment_models = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
    control_models = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
    
    print(f"\nGROUP DEFINITIONS:")
    print(f"Treatment group (Judge Score > {median_judge_score:.2f}): {len(treatment_models)} models")
    print(f"Control group (Judge Score ≤ {median_judge_score:.2f}): {len(control_models)} models")
    
    if len(treatment_models) == 0 or len(control_models) == 0:
        print("Warning: Cannot perform Bayesian analysis - insufficient group sizes")
        return {}
    
    # Extract data arrays for analysis
    treatment_errors = treatment_models['Error_Pct'].values
    control_errors = control_models['Error_Pct'].values
    treatment_success = treatment_models['Success_Rate'].values
    control_success = control_models['Success_Rate'].values
    
    bayesian_results = {}
    
    print(f"\nBAYESIAN ANALYSIS CONFIGURATION:")
    print(f"  MCMC samples: {CONFIG.get('BAYESIAN_SAMPLES', 2000):,}")
    print(f"  MCMC chains: {CONFIG.get('BAYESIAN_CHAINS', 4)}")
    print(f"  Improvement threshold: {CONFIG.get('IMPROVEMENT_THRESHOLD', 0.30):.0%}")
    print(f"  Random seed: 42 (for reproducibility)")
    
    # 1. Bayesian analysis for error reduction
    print(f"\n" + "="*50)
    print("1. BAYESIAN ANALYSIS: ESTIMATION ERROR REDUCTION")
    print("="*50)
    
    try:
        print("Building Bayesian model for error reduction...")
        
        with pm.Model() as error_model:
            # Priors for group means (weakly informative)
            mu_control = pm.Normal('mu_control', mu=50, sigma=20)  # Expected ~50% error
            mu_treatment = pm.Normal('mu_treatment', mu=50, sigma=20)
            
            # Priors for group standard deviations
            sigma_control = pm.HalfNormal('sigma_control', sigma=15)
            sigma_treatment = pm.HalfNormal('sigma_treatment', sigma=15)
            
            # Likelihood for observed data
            control_obs = pm.Normal('control_obs', mu=mu_control, sigma=sigma_control, 
                                  observed=control_errors)
            treatment_obs = pm.Normal('treatment_obs', mu=mu_treatment, sigma=sigma_treatment, 
                                    observed=treatment_errors)
            
            # Derived quantities of interest
            error_difference = pm.Deterministic('error_difference', mu_control - mu_treatment)
            error_improvement_pct = pm.Deterministic('error_improvement_pct', 
                                                   100 * error_difference / mu_control)
            
            # Probability of >30% improvement
            improvement_30_indicator = pm.Deterministic('improvement_30_plus', 
                                                      pm.math.switch(error_improvement_pct > 30, 1, 0))
            
            # Sample from posterior
            trace_error = pm.sample(
                draws=CONFIG.get('BAYESIAN_SAMPLES', 2000),
                chains=CONFIG.get('BAYESIAN_CHAINS', 4),
                tune=1000,
                target_accept=0.9,
                random_seed=42,
                return_inferencedata=True
            )
        
        # Extract posterior samples
        error_improvement_samples = trace_error.posterior['error_improvement_pct'].values.flatten()
        error_difference_samples = trace_error.posterior['error_difference'].values.flatten()
        
        # Calculate posterior statistics
        error_posterior_mean = np.mean(error_improvement_samples)
        error_posterior_std = np.std(error_improvement_samples)
        error_credible_interval = np.percentile(error_improvement_samples, [2.5, 97.5])
        
        # Probability of >30% improvement
        prob_30_plus = np.mean(error_improvement_samples > 30)
        prob_any_improvement = np.mean(error_improvement_samples > 0)
        
        # ROPE analysis (Region of Practical Equivalence: -5% to +5%)
        rope_lower, rope_upper = -5, 5
        prob_in_rope = np.mean((error_improvement_samples >= rope_lower) & 
                              (error_improvement_samples <= rope_upper))
        prob_above_rope = np.mean(error_improvement_samples > rope_upper)
        prob_below_rope = np.mean(error_improvement_samples < rope_lower)
        
        print(f"\nBayesian Results for Error Reduction:")
        print(f"  Posterior mean improvement: {error_posterior_mean:.1f}%")
        print(f"  Posterior std improvement: {error_posterior_std:.1f}%")
        print(f"  95% Credible interval: [{error_credible_interval[0]:.1f}%, {error_credible_interval[1]:.1f}%]")
        print(f"  P(improvement > 0%): {prob_any_improvement:.3f}")
        print(f"  P(improvement > 30%): {prob_30_plus:.3f}")
        print(f"  P(improvement in ROPE [-5%, +5%]): {prob_in_rope:.3f}")
        print(f"  P(improvement > ROPE): {prob_above_rope:.3f}")
        
        bayesian_results['error_reduction'] = {
            'posterior_mean': error_posterior_mean,
            'posterior_std': error_posterior_std,
            'credible_interval': error_credible_interval,
            'prob_any_improvement': prob_any_improvement,
            'prob_30_plus': prob_30_plus,
            'prob_in_rope': prob_in_rope,
            'prob_above_rope': prob_above_rope,
            'trace': trace_error,
            'posterior_samples': error_improvement_samples
        }
        
    except Exception as e:
        print(f"Error in Bayesian error reduction analysis: {e}")
        bayesian_results['error_reduction'] = {}
    
    # 2. Bayesian analysis for success rate improvement
    print(f"\n" + "="*50)
    print("2. BAYESIAN ANALYSIS: SUCCESS RATE IMPROVEMENT")
    print("="*50)
    
    try:
        print("Building Bayesian model for success rate improvement...")
        
        with pm.Model() as success_model:
            # Priors for group success rates (Beta distribution appropriate for rates)
            # Using Beta(2, 2) as weakly informative prior centered at 0.5
            p_control = pm.Beta('p_control', alpha=2, beta=2)
            p_treatment = pm.Beta('p_treatment', alpha=2, beta=2)
            
            # Transform to account for observed success rates (assume bounded [0, 1])
            success_control_scaled = control_success  # Already in [0, 1] range
            success_treatment_scaled = treatment_success
            
            # Likelihood (using Normal approximation for success rates)
            control_success_obs = pm.Normal('control_success_obs', 
                                          mu=p_control, 
                                          sigma=0.1,  # Assumed measurement error
                                          observed=success_control_scaled)
            treatment_success_obs = pm.Normal('treatment_success_obs', 
                                            mu=p_treatment, 
                                            sigma=0.1,
                                            observed=success_treatment_scaled)
            
            # Derived quantities
            success_difference = pm.Deterministic('success_difference', p_treatment - p_control)
            success_improvement_pct = pm.Deterministic('success_improvement_pct', 
                                                     100 * success_difference / p_control)
            
            # Probability of >30% improvement
            success_30_indicator = pm.Deterministic('success_30_plus', 
                                                  pm.math.switch(success_improvement_pct > 30, 1, 0))
            
            # Sample from posterior
            trace_success = pm.sample(
                draws=CONFIG.get('BAYESIAN_SAMPLES', 2000),
                chains=CONFIG.get('BAYESIAN_CHAINS', 4),
                tune=1000,
                target_accept=0.9,
                random_seed=42,
                return_inferencedata=True
            )
        
        # Extract posterior samples
        success_improvement_samples = trace_success.posterior['success_improvement_pct'].values.flatten()
        success_difference_samples = trace_success.posterior['success_difference'].values.flatten()
        
        # Calculate posterior statistics
        success_posterior_mean = np.mean(success_improvement_samples)
        success_posterior_std = np.std(success_improvement_samples)
        success_credible_interval = np.percentile(success_improvement_samples, [2.5, 97.5])
        
        # Probability calculations
        prob_success_30_plus = np.mean(success_improvement_samples > 30)
        prob_success_any_improvement = np.mean(success_improvement_samples > 0)
        
        # ROPE analysis for success rates
        success_rope_lower, success_rope_upper = -5, 5
        prob_success_in_rope = np.mean((success_improvement_samples >= success_rope_lower) & 
                                     (success_improvement_samples <= success_rope_upper))
        prob_success_above_rope = np.mean(success_improvement_samples > success_rope_upper)
        
        print(f"\nBayesian Results for Success Rate Improvement:")
        print(f"  Posterior mean improvement: {success_posterior_mean:.1f}%")
        print(f"  Posterior std improvement: {success_posterior_std:.1f}%")
        print(f"  95% Credible interval: [{success_credible_interval[0]:.1f}%, {success_credible_interval[1]:.1f}%]")
        print(f"  P(improvement > 0%): {prob_success_any_improvement:.3f}")
        print(f"  P(improvement > 30%): {prob_success_30_plus:.3f}")
        print(f"  P(improvement in ROPE [-5%, +5%]): {prob_success_in_rope:.3f}")
        print(f"  P(improvement > ROPE): {prob_success_above_rope:.3f}")
        
        bayesian_results['success_improvement'] = {
            'posterior_mean': success_posterior_mean,
            'posterior_std': success_posterior_std,
            'credible_interval': success_credible_interval,
            'prob_any_improvement': prob_success_any_improvement,
            'prob_30_plus': prob_success_30_plus,
            'prob_in_rope': prob_success_in_rope,
            'prob_above_rope': prob_success_above_rope,
            'trace': trace_success,
            'posterior_samples': success_improvement_samples
        }
        
    except Exception as e:
        print(f"Error in Bayesian success rate analysis: {e}")
        bayesian_results['success_improvement'] = {}
    
    # 3. Bayesian Model Comparison using WAIC
    print(f"\n" + "="*50)
    print("3. BAYESIAN MODEL COMPARISON")
    print("="*50)
    
    try:
        print("Comparing models with and without treatment effect...")
        
        # Model with treatment effect (already computed above)
        if 'error_reduction' in bayesian_results and bayesian_results['error_reduction']:
            trace_with_effect = bayesian_results['error_reduction']['trace']
            
            # Null model (no treatment effect)
            with pm.Model() as null_model:
                # Single population model
                mu_pooled = pm.Normal('mu_pooled', mu=50, sigma=20)
                sigma_pooled = pm.HalfNormal('sigma_pooled', sigma=15)
                
                # All observations from same distribution
                all_errors = np.concatenate([control_errors, treatment_errors])
                all_obs = pm.Normal('all_obs', mu=mu_pooled, sigma=sigma_pooled, 
                                  observed=all_errors)
                
                trace_null = pm.sample(
                    draws=CONFIG.get('BAYESIAN_SAMPLES', 2000),
                    chains=CONFIG.get('BAYESIAN_CHAINS', 4),
                    tune=1000,
                    random_seed=42,
                    return_inferencedata=True
                )
            
            # Calculate WAIC for model comparison
            waic_with_effect = az.waic(trace_with_effect)
            waic_null = az.waic(trace_null)
            
            # Bayes Factor approximation using WAIC
            delta_waic = waic_with_effect.waic - waic_null.waic
            
            print(f"Model Comparison Results:")
            print(f"  Treatment effect model WAIC: {waic_with_effect.waic:.2f} ± {waic_with_effect.waic_se:.2f}")
            print(f"  Null model WAIC: {waic_null.waic:.2f} ± {waic_null.waic_se:.2f}")
            print(f"  ΔWAIC (effect - null): {delta_waic:.2f}")
            
            if delta_waic < -2:
                model_preference = "Strong evidence for treatment effect"
            elif delta_waic < 0:
                model_preference = "Moderate evidence for treatment effect"
            elif delta_waic < 2:
                model_preference = "Weak evidence either way"
            else:
                model_preference = "Evidence against treatment effect"
            
            print(f"  Model preference: {model_preference}")
            
            bayesian_results['model_comparison'] = {
                'waic_with_effect': waic_with_effect.waic,
                'waic_null': waic_null.waic,
                'delta_waic': delta_waic,
                'preference': model_preference
            }
        
    except Exception as e:
        print(f"Error in Bayesian model comparison: {e}")
        bayesian_results['model_comparison'] = {}
    
    # 4. Posterior Predictive Checks
    print(f"\n" + "="*50)
    print("4. POSTERIOR PREDICTIVE CHECKS")
    print("="*50)
    
    try:
        if 'error_reduction' in bayesian_results and bayesian_results['error_reduction']:
            trace_error = bayesian_results['error_reduction']['trace']
            
            # Generate posterior predictive samples
            with error_model:
                ppc = pm.sample_posterior_predictive(trace_error, random_seed=42)
            
            # Compare observed vs predicted statistics
            obs_control_mean = np.mean(control_errors)
            obs_treatment_mean = np.mean(treatment_errors)
            obs_difference = obs_control_mean - obs_treatment_mean
            
            pred_control_means = np.mean(ppc.posterior_predictive['control_obs'], axis=(0, 1, 2))
            pred_treatment_means = np.mean(ppc.posterior_predictive['treatment_obs'], axis=(0, 1, 2))
            pred_differences = pred_control_means - pred_treatment_means
            
            # Calculate Bayesian p-values
            p_value_control = np.mean(np.abs(pred_control_means - obs_control_mean) >= 
                                    np.abs(pred_control_means - obs_control_mean))
            p_value_treatment = np.mean(np.abs(pred_treatment_means - obs_treatment_mean) >= 
                                      np.abs(pred_treatment_means - obs_treatment_mean))
            p_value_difference = np.mean(pred_differences >= obs_difference)
            
            print(f"Posterior Predictive Check Results:")
            print(f"  Observed control mean: {obs_control_mean:.2f}%")
            print(f"  Predicted control mean: {np.mean(pred_control_means):.2f}% ± {np.std(pred_control_means):.2f}")
            print(f"  Observed treatment mean: {obs_treatment_mean:.2f}%")
            print(f"  Predicted treatment mean: {np.mean(pred_treatment_means):.2f}% ± {np.std(pred_treatment_means):.2f}")
            print(f"  Bayesian p-value (difference): {p_value_difference:.3f}")
            
            bayesian_results['posterior_predictive'] = {
                'obs_control_mean': obs_control_mean,
                'obs_treatment_mean': obs_treatment_mean,
                'pred_control_means': pred_control_means,
                'pred_treatment_means': pred_treatment_means,
                'p_value_difference': p_value_difference
            }
        
    except Exception as e:
        print(f"Error in posterior predictive checks: {e}")
        bayesian_results['posterior_predictive'] = {}
    
    # 5. Bayesian Decision Analysis
    print(f"\n" + "="*50)
    print("5. BAYESIAN DECISION ANALYSIS")
    print("="*50)
    
    try:
        if ('error_reduction' in bayesian_results and bayesian_results['error_reduction'] and
            'success_improvement' in bayesian_results and bayesian_results['success_improvement']):
            
            error_samples = bayesian_results['error_reduction']['posterior_samples']
            success_samples = bayesian_results['success_improvement']['posterior_samples']
            
            # Combined probability of achieving >30% in either metric
            prob_either_30 = np.mean((error_samples > 30) | (success_samples > 30))
            
            # Combined probability of achieving >30% in both metrics
            prob_both_30 = np.mean((error_samples > 30) & (success_samples > 30))
            
            # Expected value of improvement
            expected_error_improvement = np.mean(error_samples)
            expected_success_improvement = np.mean(success_samples)
            
            # Risk assessment (probability of negative outcomes)
            prob_error_worse = np.mean(error_samples < -10)  # >10% worse performance
            prob_success_worse = np.mean(success_samples < -10)
            
            print(f"Bayesian Decision Analysis:")
            print(f"  P(>30% improvement in either metric): {prob_either_30:.3f}")
            print(f"  P(>30% improvement in both metrics): {prob_both_30:.3f}")
            print(f"  Expected error improvement: {expected_error_improvement:.1f}%")
            print(f"  Expected success improvement: {expected_success_improvement:.1f}%")
            print(f"  P(>10% worse error performance): {prob_error_worse:.3f}")
            print(f"  P(>10% worse success performance): {prob_success_worse:.3f}")
            
            # Decision recommendation
            if prob_either_30 > 0.8:
                decision = "STRONG RECOMMENDATION: Adopt hallucination-reducing techniques"
            elif prob_either_30 > 0.6:
                decision = "MODERATE RECOMMENDATION: Consider adoption with monitoring"
            elif prob_either_30 > 0.4:
                decision = "WEAK RECOMMENDATION: Proceed with caution"
            else:
                decision = "NOT RECOMMENDED: Insufficient evidence of benefit"
            
            print(f"\nDecision Recommendation: {decision}")
            
            bayesian_results['decision_analysis'] = {
                'prob_either_30': prob_either_30,
                'prob_both_30': prob_both_30,
                'expected_error_improvement': expected_error_improvement,
                'expected_success_improvement': expected_success_improvement,
                'prob_error_worse': prob_error_worse,
                'prob_success_worse': prob_success_worse,
                'recommendation': decision
            }
        
    except Exception as e:
        print(f"Error in Bayesian decision analysis: {e}")
        bayesian_results['decision_analysis'] = {}
    
    # 6. Bayesian Analysis Summary
    print(f"\n" + "="*60)
    print("BAYESIAN ANALYSIS SUMMARY")
    print("="*60)
    
    print(f"\nKey Posterior Probabilities:")
    if 'error_reduction' in bayesian_results and bayesian_results['error_reduction']:
        print(f"  P(Error reduction > 30%): {bayesian_results['error_reduction']['prob_30_plus']:.3f}")
    if 'success_improvement' in bayesian_results and bayesian_results['success_improvement']:
        print(f"  P(Success improvement > 30%): {bayesian_results['success_improvement']['prob_30_plus']:.3f}")
    if 'decision_analysis' in bayesian_results and bayesian_results['decision_analysis']:
        print(f"  P(Either metric > 30%): {bayesian_results['decision_analysis']['prob_either_30']:.3f}")
    
    print(f"\nModel Evidence:")
    if 'model_comparison' in bayesian_results and bayesian_results['model_comparison']:
        print(f"  {bayesian_results['model_comparison']['preference']}")
        print(f"  ΔWAIC: {bayesian_results['model_comparison']['delta_waic']:.2f}")
    
    print(f"\nADVANTAGES OF BAYESIAN ANALYSIS:")
    print(f"  ✓ Direct probability statements about hypotheses")
    print(f"  ✓ Credible intervals with intuitive interpretation")
    print(f"  ✓ Incorporates prior knowledge and uncertainty")
    print(f"  ✓ Model comparison and selection capabilities")
    print(f"  ✓ Decision-theoretic framework for recommendations")
    
    return bayesian_results

# Execute Bayesian analysis
if ('performance_df' in globals() and not performance_df.empty and 
    STATISTICAL_CONFIG.get('bayesian_analysis', False)):
    bayesian_results = perform_bayesian_analysis()
    print(f"\n✓ Bayesian analysis completed successfully")
else:
    print("Cannot perform Bayesian analysis:")
    if 'performance_df' not in globals() or performance_df.empty:
        print("  - Performance data not available")
    if not STATISTICAL_CONFIG.get('bayesian_analysis', False):
        print("  - Bayesian analysis functionality not available")
        print("  - Install with: pip install 'praxis-requirements-analyzer[dev]'")
    
    bayesian_results = {}

In [ ]:
# Cell [12] - Enhanced Statistical Validation Visualizations with Advanced Methods
# Purpose: Create comprehensive visualizations for all statistical tests including permutation, bootstrap, and Bayesian results
# Dependencies: statistical_results, permutation_results, bootstrap_results, bayesian_results from previous cells
# Breadcrumbs: Setup -> Analysis -> Advanced Statistical Testing -> Enhanced Validation Visualizations

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap
import numpy as np

def create_enhanced_statistical_visualizations():
    """
    Create comprehensive visualizations to support all statistical hypothesis testing methods
    """
    
    if performance_df.empty:
        print("No data available for statistical visualizations")
        return
    
    # Set up the enhanced figure with more subplots
    fig = plt.figure(figsize=(24, 20), constrained_layout=True)  # Use constrained_layout instead
    fig.suptitle('Enhanced Statistical Validation of Hallucination-Reducing Techniques\n' +
                 'Hypothesis: >30% Improvement in Time-to-Market\n' +
                 'Methods: Classical, Permutation, Bootstrap, and Bayesian Analysis', fontsize=18)
    
    # Create a more complex grid layout
    gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
    
    # Define groups based on judge scores (using median split)
    median_judge_score = performance_df['Avg_Judge_Score'].median()
    treatment_group = performance_df[performance_df['Avg_Judge_Score'] > median_judge_score]
    control_group = performance_df[performance_df['Avg_Judge_Score'] <= median_judge_score]
    
    # 1. Enhanced Box plot comparison with all test results
    ax1 = fig.add_subplot(gs[0, 0])
    data_for_box = [control_group['Error_Pct'].values, treatment_group['Error_Pct'].values]
    box_plot = ax1.boxplot(data_for_box, labels=['Control\n(Low Judge Score)', 'Treatment\n(High Judge Score)'], 
                          patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightcoral')
    box_plot['boxes'][1].set_facecolor('lightgreen')
    ax1.set_ylabel('Estimation Error (%)')
    ax1.set_title('Error Rate Distribution by Group\nwith Statistical Test Results')
    ax1.grid(True, alpha=0.3)
    
    # Add comprehensive significance indicators
    test_results_text = ""
    if 'statistical_results' in globals() and 't_test' in statistical_results:
        p_val = statistical_results['t_test']['p_value']
        test_results_text += f"t-test: p={p_val:.4f}\n"
    if 'permutation_results' in globals() and 'error_reduction' in permutation_results:
        perm_p = permutation_results['error_reduction'].get('p_value', 'N/A')
        test_results_text += f"Permutation: p={perm_p:.4f}\n" if perm_p != 'N/A' else ""
    if 'bootstrap_results' in globals() and 'error_reduction' in bootstrap_results:
        boot_p = bootstrap_results['error_reduction'].get('p_value_improvement', 'N/A')
        test_results_text += f"Bootstrap: p={boot_p:.4f}\n" if boot_p != 'N/A' else ""
    if 'bayesian_results' in globals() and 'error_reduction' in bayesian_results:
        bayes_prob = bayesian_results['error_reduction'].get('prob_30_plus', 'N/A')
        test_results_text += f"Bayesian P(>30%): {bayes_prob:.3f}" if bayes_prob != 'N/A' else ""
    
    ax1.text(0.02, 0.98, test_results_text, transform=ax1.transAxes, 
             verticalalignment='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 2. Permutation Test Results
    ax2 = fig.add_subplot(gs[0, 1])
    if 'permutation_results' in globals() and 'error_reduction' in permutation_results:
        perm_data = permutation_results['error_reduction']
        if 'null_distribution' in perm_data:
            null_dist = perm_data['null_distribution']
            observed_stat = perm_data['observed_statistic']
            
            ax2.hist(null_dist, bins=50, alpha=0.7, color='lightblue', density=True, 
                    label='Null Distribution')
            ax2.axvline(observed_stat, color='red', linestyle='--', linewidth=2, 
                       label=f'Observed: {observed_stat:.2f}')
            ax2.set_xlabel('Error Difference (Control - Treatment)')
            ax2.set_ylabel('Density')
            ax2.set_title('Permutation Test: Error Reduction\nNull Distribution vs Observed')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Add p-value annotation
            p_val = perm_data.get('p_value', 'N/A')
            ax2.text(0.95, 0.95, f'p = {p_val:.4f}', transform=ax2.transAxes, 
                    ha='right', va='top', fontsize=12,
                    bbox=dict(boxstyle='round', facecolor='yellow' if p_val < 0.05 else 'white'))
    else:
        ax2.text(0.5, 0.5, 'Permutation Test\nNot Available', ha='center', va='center', 
                transform=ax2.transAxes, fontsize=14)
        ax2.set_title('Permutation Test Results')
    
    # 3. Bootstrap Confidence Intervals
    ax3 = fig.add_subplot(gs[0, 2])
    if 'bootstrap_results' in globals():
        models = []
        improvements = []
        ci_lowers = []
        ci_uppers = []
        
        if 'error_reduction' in bootstrap_results:
            models.append('Error\nReduction')
            improvements.append(bootstrap_results['error_reduction'].get('observed_improvement', 0))
            ci = bootstrap_results['error_reduction'].get('confidence_interval', (0, 0))
            ci_lowers.append(ci[0])
            ci_uppers.append(ci[1])
        
        if 'success_improvement' in bootstrap_results:
            models.append('Success\nImprovement')
            improvements.append(bootstrap_results['success_improvement'].get('observed_improvement', 0))
            ci = bootstrap_results['success_improvement'].get('confidence_interval', (0, 0))
            ci_lowers.append(ci[0])
            ci_uppers.append(ci[1])
        
        if models:
            x_pos = np.arange(len(models))
            bars = ax3.bar(x_pos, improvements, alpha=0.7, 
                          color=['green' if imp > 30 else 'orange' if imp > 0 else 'red' 
                                for imp in improvements])
            
            # Add error bars for confidence intervals
            errors = [[imp - ci_low for imp, ci_low in zip(improvements, ci_lowers)],
                     [ci_up - imp for imp, ci_up in zip(improvements, ci_uppers)]]
            ax3.errorbar(x_pos, improvements, yerr=errors, fmt='none', 
                        color='black', capsize=5, capthick=2)
            
            ax3.axhline(y=30, color='red', linestyle='--', linewidth=2, label='30% Threshold')
            ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
            ax3.set_xticks(x_pos)
            ax3.set_xticklabels(models)
            ax3.set_ylabel('Improvement (%)')
            ax3.set_title('Bootstrap Confidence Intervals\nfor Improvement Metrics')
            ax3.legend()
            ax3.grid(True, alpha=0.3)
        else:
            ax3.text(0.5, 0.5, 'Bootstrap Results\nNot Available', ha='center', va='center',
                    transform=ax3.transAxes, fontsize=14)
    else:
        ax3.text(0.5, 0.5, 'Bootstrap Results\nNot Available', ha='center', va='center',
                transform=ax3.transAxes, fontsize=14)
        ax3.set_title('Bootstrap Confidence Intervals')
    
    # 4. Bayesian Posterior Distributions
    ax4 = fig.add_subplot(gs[0, 3])
    if 'bayesian_results' in globals():
        if 'error_reduction' in bayesian_results and 'posterior_samples' in bayesian_results['error_reduction']:
            error_samples = bayesian_results['error_reduction']['posterior_samples']
            ax4.hist(error_samples, bins=50, alpha=0.7, color='skyblue', density=True,
                    label='Error Reduction')
            
            # Add credible interval
            ci = bayesian_results['error_reduction'].get('credible_interval', [0, 0])
            ax4.axvline(ci[0], color='blue', linestyle=':', alpha=0.7, label=f'95% CI')
            ax4.axvline(ci[1], color='blue', linestyle=':', alpha=0.7)
            
        if 'success_improvement' in bayesian_results and 'posterior_samples' in bayesian_results['success_improvement']:
            success_samples = bayesian_results['success_improvement']['posterior_samples']
            ax4.hist(success_samples, bins=50, alpha=0.7, color='lightgreen', density=True,
                    label='Success Improvement')
        
        ax4.axvline(30, color='red', linestyle='--', linewidth=2, label='30% Threshold')
        ax4.axvline(0, color='black', linestyle='-', linewidth=1)
        ax4.set_xlabel('Improvement (%)')
        ax4.set_ylabel('Posterior Density')
        ax4.set_title('Bayesian Posterior Distributions')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # Add probability annotations
        if 'error_reduction' in bayesian_results:
            prob_30 = bayesian_results['error_reduction'].get('prob_30_plus', 0)
            ax4.text(0.02, 0.95, f'P(Error >30%): {prob_30:.3f}', 
                    transform=ax4.transAxes, va='top', fontsize=10,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        ax4.text(0.5, 0.5, 'Bayesian Results\nNot Available', ha='center', va='center',
                transform=ax4.transAxes, fontsize=14)
        ax4.set_title('Bayesian Posterior Distributions')
    
    # 5. Model Performance Scatter with Advanced Statistics
    ax5 = fig.add_subplot(gs[1, :2])
    scatter = ax5.scatter(performance_df['Avg_Judge_Score'], performance_df['Error_Pct'], 
                         c=performance_df['Success_Rate'], cmap='RdYlGn', s=150, alpha=0.8)
    
    # Add trend line
    z = np.polyfit(performance_df['Avg_Judge_Score'], performance_df['Error_Pct'], 1)
    p = np.poly1d(z)
    ax5.plot(performance_df['Avg_Judge_Score'], p(performance_df['Avg_Judge_Score']), 
             "r--", alpha=0.8, linewidth=2)
    
    # Add median split line
    ax5.axvline(median_judge_score, color='purple', linestyle=':', linewidth=2, 
               label=f'Median Split: {median_judge_score:.2f}')
    
    # Enhanced correlation and statistical info
    corr_coef = performance_df['Avg_Judge_Score'].corr(performance_df['Error_Pct'])
    stats_text = f'Correlation: r = {corr_coef:.3f}\n'
    
    if 'statistical_results' in globals() and 'effect_size' in statistical_results:
        cohens_d = statistical_results['effect_size'].get('cohens_d', 'N/A')
        stats_text += f"Cohen's d: {cohens_d:.3f}\n" if cohens_d != 'N/A' else ""
    
    if 'bayesian_results' in globals() and 'model_comparison' in bayesian_results:
        delta_waic = bayesian_results['model_comparison'].get('delta_waic', 'N/A')
        stats_text += f"ΔWAIC: {delta_waic:.2f}" if delta_waic != 'N/A' else ""
    
    ax5.text(0.02, 0.98, stats_text, transform=ax5.transAxes, 
             verticalalignment='top', fontsize=11,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax5.set_xlabel('Average Judge Score')
    ax5.set_ylabel('Estimation Error (%)')
    ax5.set_title('Model Performance: Judge Score vs Accuracy\nwith Statistical Relationship Analysis')
    ax5.legend()
    
    # Create colorbar separately to avoid layout issues
    cbar = fig.colorbar(scatter, ax=ax5, label='Success Rate', shrink=0.8)
    ax5.grid(True, alpha=0.3)
    
    # 6. Statistical Methods Comparison Table
    ax6 = fig.add_subplot(gs[1, 2:])
    ax6.axis('tight')
    ax6.axis('off')
    
    # Create comprehensive comparison table
    methods = ['Classical t-test', 'Permutation Test', 'Bootstrap Test', 'Bayesian Analysis']
    metrics = ['p-value/Probability', 'Effect Size', 'Confidence/Credible Interval', 'Interpretation']
    
    table_data = []
    
    # Classical t-test
    if 'statistical_results' in globals() and 't_test' in statistical_results:
        t_result = statistical_results['t_test']
        classical_row = [
            f"{t_result.get('p_value', 'N/A'):.4f}" if t_result.get('p_value') != 'N/A' else 'N/A',
            f"{statistical_results.get('effect_size', {}).get('cohens_d', 'N/A'):.3f}" if 'effect_size' in statistical_results else 'N/A',
            f"[{t_result.get('ci_95', [0,0])[0]:.2f}, {t_result.get('ci_95', [0,0])[1]:.2f}]" if 'ci_95' in t_result else 'N/A',
            'Significant' if t_result.get('significant', False) else 'Not Significant'
        ]
    else:
        classical_row = ['N/A', 'N/A', 'N/A', 'N/A']
    table_data.append(classical_row)
    
    # Permutation test
    if 'permutation_results' in globals() and 'error_reduction' in permutation_results:
        perm_result = permutation_results['error_reduction']
        perm_row = [
            f"{perm_result.get('p_value', 'N/A'):.4f}" if perm_result.get('p_value') != 'N/A' else 'N/A',
            f"{perm_result.get('improvement_pct', 'N/A'):.1f}%" if perm_result.get('improvement_pct') != 'N/A' else 'N/A',
            'Exact CI Available' if 'confidence_intervals' in permutation_results else 'N/A',
            'Significant' if perm_result.get('significant', False) else 'Not Significant'
        ]
    else:
        perm_row = ['N/A', 'N/A', 'N/A', 'N/A']
    table_data.append(perm_row)
    
    # Bootstrap test
    if 'bootstrap_results' in globals() and 'error_reduction' in bootstrap_results:
        boot_result = bootstrap_results['error_reduction']
        boot_row = [
            f"{boot_result.get('p_value_30_percent', 'N/A'):.4f}" if boot_result.get('p_value_30_percent') != 'N/A' else 'N/A',
            f"{boot_result.get('observed_improvement', 'N/A'):.1f}%" if boot_result.get('observed_improvement') != 'N/A' else 'N/A',
            f"[{boot_result.get('confidence_interval', [0,0])[0]:.1f}, {boot_result.get('confidence_interval', [0,0])[1]:.1f}]" if 'confidence_interval' in boot_result else 'N/A',
            'Meets Threshold' if boot_result.get('meets_threshold', False) else 'Below Threshold'
        ]
    else:
        boot_row = ['N/A', 'N/A', 'N/A', 'N/A']
    table_data.append(boot_row)
    
    # Bayesian analysis
    if 'bayesian_results' in globals() and 'error_reduction' in bayesian_results:
        bayes_result = bayesian_results['error_reduction']
        bayes_row = [
            f"{bayes_result.get('prob_30_plus', 'N/A'):.3f}" if bayes_result.get('prob_30_plus') != 'N/A' else 'N/A',
            f"{bayes_result.get('posterior_mean', 'N/A'):.1f}%" if bayes_result.get('posterior_mean') != 'N/A' else 'N/A',
            f"[{bayes_result.get('credible_interval', [0,0])[0]:.1f}, {bayes_result.get('credible_interval', [0,0])[1]:.1f}]" if 'credible_interval' in bayes_result else 'N/A',
            'Strong Evidence' if bayes_result.get('prob_30_plus', 0) > 0.8 else 'Moderate Evidence' if bayes_result.get('prob_30_plus', 0) > 0.6 else 'Weak Evidence'
        ]
    else:
        bayes_row = ['N/A', 'N/A', 'N/A', 'N/A']
    table_data.append(bayes_row)
    
    table = ax6.table(cellText=table_data,
                     rowLabels=methods,
                     colLabels=metrics,
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 2)
    
    # Color code the table based on results
    for i in range(len(methods)):
        for j in range(len(metrics)):
            cell = table[(i+1, j)]
            if 'Significant' in table_data[i][j] or 'Strong Evidence' in table_data[i][j]:
                cell.set_facecolor('lightgreen')
            elif 'Not Significant' in table_data[i][j] or 'Weak Evidence' in table_data[i][j]:
                cell.set_facecolor('lightcoral')
            elif 'Moderate' in table_data[i][j]:
                cell.set_facecolor('lightyellow')
    
    ax6.set_title('Statistical Methods Comparison Summary', pad=20, fontsize=14)
    
    # 7-10. Bootstrap and Bayesian Distribution Details (if available)
    methods_available = 0
    
    # Bootstrap distributions
    if 'bootstrap_results' in globals():
        for idx, (test_name, label) in enumerate([('error_reduction', 'Error Reduction'), 
                                                 ('success_improvement', 'Success Improvement')]):
            if test_name in bootstrap_results and 'bootstrap_distribution' in bootstrap_results[test_name]:
                ax = fig.add_subplot(gs[2, methods_available])
                
                boot_dist = bootstrap_results[test_name]['bootstrap_distribution']
                observed = bootstrap_results[test_name]['observed_improvement']
                ci = bootstrap_results[test_name]['confidence_interval']
                
                ax.hist(boot_dist, bins=40, alpha=0.7, color='steelblue', density=True)
                ax.axvline(observed, color='red', linestyle='-', linewidth=2, 
                          label=f'Observed: {observed:.1f}%')
                ax.axvline(ci[0], color='orange', linestyle='--', alpha=0.7)
                ax.axvline(ci[1], color='orange', linestyle='--', alpha=0.7, 
                          label=f'95% CI: [{ci[0]:.1f}, {ci[1]:.1f}]')
                ax.axvline(30, color='green', linestyle=':', linewidth=2, label='30% Threshold')
                
                ax.set_xlabel('Improvement (%)')
                ax.set_ylabel('Bootstrap Density')
                ax.set_title(f'Bootstrap Distribution\n{label}')
                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)
                
                methods_available += 1
                if methods_available >= 2:
                    break
    
    # Bayesian posterior distributions (detailed)
    if 'bayesian_results' in globals() and methods_available < 4:
        for idx, (test_name, label) in enumerate([('error_reduction', 'Error Reduction'), 
                                                 ('success_improvement', 'Success Improvement')]):
            if (test_name in bayesian_results and 'posterior_samples' in bayesian_results[test_name] 
                and methods_available < 4):
                ax = fig.add_subplot(gs[2, methods_available])
                
                samples = bayesian_results[test_name]['posterior_samples']
                ci = bayesian_results[test_name]['credible_interval']
                prob_30 = bayesian_results[test_name]['prob_30_plus']
                
                ax.hist(samples, bins=40, alpha=0.7, color='lightcoral', density=True)
                ax.axvline(ci[0], color='blue', linestyle='--', alpha=0.7)
                ax.axvline(ci[1], color='blue', linestyle='--', alpha=0.7, 
                          label=f'95% Credible: [{ci[0]:.1f}, {ci[1]:.1f}]')
                ax.axvline(30, color='green', linestyle=':', linewidth=2, label='30% Threshold')
                
                # Shade area above 30%
                x_range = np.linspace(samples.min(), samples.max(), 1000)
                density = stats.gaussian_kde(samples)(x_range)
                ax.fill_between(x_range, 0, density, where=(x_range >= 30), 
                               alpha=0.3, color='green', label=f'P(>30%) = {prob_30:.3f}')
                
                ax.set_xlabel('Improvement (%)')
                ax.set_ylabel('Posterior Density')
                ax.set_title(f'Bayesian Posterior\n{label}')
                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)
                
                methods_available += 1
                if methods_available >= 4:
                    break
    
    # 11. Overall Statistical Evidence Summary
    ax11 = fig.add_subplot(gs[3, :])
    ax11.axis('off')
    
    # Create evidence summary
    evidence_summary = "COMPREHENSIVE STATISTICAL EVIDENCE SUMMARY\n" + "="*80 + "\n\n"
    
    # Classical statistics
    if 'statistical_results' in globals() and 't_test' in statistical_results:
        t_sig = statistical_results['t_test'].get('significant', False)
        evidence_summary += f"📊 CLASSICAL STATISTICS: {'SIGNIFICANT' if t_sig else 'NOT SIGNIFICANT'}\n"
        evidence_summary += f"   t-test p-value: {statistical_results['t_test'].get('p_value', 'N/A'):.4f}\n"
        if 'effect_size' in statistical_results:
            effect_size = statistical_results['effect_size'].get('cohens_d', 'N/A')
            evidence_summary += f"   Effect size (Cohen's d): {effect_size:.3f} ({statistical_results['effect_size'].get('interpretation', 'N/A')} effect)\n"
    
    evidence_summary += "\n"
    
    # Permutation tests
    if 'permutation_results' in globals():
        perm_sig_count = sum(1 for test in ['error_reduction', 'success_improvement', 'combined_improvement'] 
                           if test in permutation_results and permutation_results[test].get('significant', False))
        perm_total = len([test for test in ['error_reduction', 'success_improvement', 'combined_improvement'] 
                         if test in permutation_results])
        evidence_summary += f"🔄 PERMUTATION TESTS: {perm_sig_count}/{perm_total} SIGNIFICANT\n"
        if 'error_reduction' in permutation_results:
            evidence_summary += f"   Error reduction p-value: {permutation_results['error_reduction'].get('p_value', 'N/A'):.4f}\n"
        if 'success_improvement' in permutation_results:
            evidence_summary += f"   Success improvement p-value: {permutation_results['success_improvement'].get('p_value', 'N/A'):.4f}\n"
    
    evidence_summary += "\n"
    
    # Bootstrap results
    if 'bootstrap_results' in globals():
        boot_threshold_count = sum(1 for test in ['error_reduction', 'success_improvement'] 
                                 if test in bootstrap_results and bootstrap_results[test].get('meets_threshold', False))
        boot_total = len([test for test in ['error_reduction', 'success_improvement'] 
                         if test in bootstrap_results])
        evidence_summary += f"🔀 BOOTSTRAP ANALYSIS: {boot_threshold_count}/{boot_total} MEET >30% THRESHOLD\n"
        if 'error_reduction' in bootstrap_results:
            conf = bootstrap_results['error_reduction'].get('threshold_confidence', 0)
            evidence_summary += f"   Error reduction confidence for 30%: {conf:.3f}\n"
        if 'success_improvement' in bootstrap_results:
            conf = bootstrap_results['success_improvement'].get('threshold_confidence', 0)
            evidence_summary += f"   Success improvement confidence for 30%: {conf:.3f}\n"
    
    evidence_summary += "\n"
    
    # Bayesian results
    if 'bayesian_results' in globals():
        evidence_summary += f"🎯 BAYESIAN ANALYSIS: POSTERIOR PROBABILITIES\n"
        if 'error_reduction' in bayesian_results:
            prob = bayesian_results['error_reduction'].get('prob_30_plus', 0)
            evidence_summary += f"   P(Error reduction > 30%): {prob:.3f}\n"
        if 'success_improvement' in bayesian_results:
            prob = bayesian_results['success_improvement'].get('prob_30_plus', 0)
            evidence_summary += f"   P(Success improvement > 30%): {prob:.3f}\n"
        if 'decision_analysis' in bayesian_results:
            prob_either = bayesian_results['decision_analysis'].get('prob_either_30', 0)
            evidence_summary += f"   P(Either metric > 30%): {prob_either:.3f}\n"
            recommendation = bayesian_results['decision_analysis'].get('recommendation', 'N/A')
            evidence_summary += f"   Decision: {recommendation}\n"
    
    evidence_summary += "\n" + "="*80 + "\n"
    evidence_summary += "🎯 OVERALL CONCLUSION: "
    
    # Determine overall conclusion
    significant_methods = 0
    total_methods = 0
    
    if 'statistical_results' in globals() and 't_test' in statistical_results:
        total_methods += 1
        if statistical_results['t_test'].get('significant', False):
            significant_methods += 1
    
    if 'permutation_results' in globals() and 'error_reduction' in permutation_results:
        total_methods += 1
        if permutation_results['error_reduction'].get('significant', False):
            significant_methods += 1
    
    if 'bootstrap_results' in globals() and 'error_reduction' in bootstrap_results:
        total_methods += 1
        if bootstrap_results['error_reduction'].get('meets_threshold', False):
            significant_methods += 1
    
    if 'bayesian_results' in globals() and 'error_reduction' in bayesian_results:
        total_methods += 1
        if bayesian_results['error_reduction'].get('prob_30_plus', 0) > 0.8:
            significant_methods += 1
    
    if significant_methods >= total_methods * 0.75:
        conclusion = "STRONG EVIDENCE for >30% improvement hypothesis"
    elif significant_methods >= total_methods * 0.5:
        conclusion = "MODERATE EVIDENCE for >30% improvement hypothesis"
    else:
        conclusion = "WEAK EVIDENCE for >30% improvement hypothesis"
    
    evidence_summary += conclusion
    
    ax11.text(0.05, 0.95, evidence_summary, transform=ax11.transAxes, 
              fontsize=11, verticalalignment='top', fontfamily='monospace',
              bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # Use try/except for layout to handle potential colorbar conflicts
    try:
        # Don't use tight_layout, rely on constrained_layout set at figure creation
        pass
    except Exception as e:
        print(f"Layout adjustment warning: {e}")
    
    plt.show()
    
    print("✓ Enhanced statistical visualizations completed successfully")
    print(f"✓ Integrated results from {total_methods} statistical methods")
    print(f"✓ {significant_methods}/{total_methods} methods show positive evidence")

# Execute enhanced statistical visualizations
if 'performance_df' in globals() and not performance_df.empty:
    create_enhanced_statistical_visualizations()
else:
    print("Cannot create enhanced statistical visualizations - performance data not available")

In [ ]:
# Cell [13] - Enhanced Final Executive Summary with Advanced Statistical Analysis
# Purpose: Generate comprehensive executive summary integrating all statistical methods and business insights
# Dependencies: All previous analysis results, statistical_results, permutation_results, bootstrap_results, bayesian_results
# Breadcrumbs: Setup -> Analysis -> Advanced Statistical Testing -> Enhanced Final Executive Summary & Business Impact Report

print("ENHANCED EXECUTIVE SUMMARY - COMPREHENSIVE STATISTICAL ANALYSIS")
print("=" * 70)

# Get project name safely
project_name = CONFIG.get('NEO4J_PROJECT_NAME', 'Unknown Project') if 'CONFIG' in globals() else 'Unknown Project'
print(f"Project: {project_name}")
print(f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")
print(f"Analysis Framework: Classical, Permutation, Bootstrap, and Bayesian Methods")

if 'performance_df' in globals() and not performance_df.empty:
    print(f"\n" + "="*70)
    print("DATA SUMMARY")
    print("="*70)
    
    if 'code_metrics_df' in globals():
        print(f"  Total code files in project: {len(code_metrics_df)}")
        print(f"  Total lines of code in project: {code_metrics_df['CountLineCode'].sum():,}")
    else:
        print("  Code metrics: Not available")
    
    if 'ground_truth_requirements' in globals():
        print(f"  Ground truth requirements: {len(ground_truth_requirements)}")
    else:
        print("  Ground truth requirements: Not available")
    
    if 'llm_estimates_df' in globals() and not llm_estimates_df.empty:
        estimated_requirements = llm_estimates_df['requirement_id'].unique()
        if 'ground_truth_requirements' in globals():
            print(f"  Requirements with estimates: {len(estimated_requirements)} ({len(estimated_requirements)/len(ground_truth_requirements):.1%})")
        else:
            print(f"  Requirements with estimates: {len(estimated_requirements)}")
    
    # Get baseline values safely
    conversion_factor = CONFIG.get('CONVERSION_FACTOR', 0.957) if 'CONFIG' in globals() else 0.957
    industry_loc_per_sifp = industry_metrics.get('LOC_PER_SIFP', 100) if 'industry_metrics' in globals() else 100
    project_loc_per_sifp = baseline_metrics.get('project_loc_per_sifp', industry_loc_per_sifp) if 'baseline_metrics' in globals() else industry_loc_per_sifp
    desharnais_hours_per_sifp = effort_metrics.get('avg_hours_per_sifp', 10) if 'effort_metrics' in globals() else 10
    
    print(f"\nConversion Factors and Baselines:")
    print(f"  UFP → SiFP: {conversion_factor} (from Desharnais research)")
    print(f"  SiFP → Effort: {desharnais_hours_per_sifp:.2f} hours/SiFP (Desharnais dataset)")
    print(f"  SiFP → LOC: {project_loc_per_sifp:.1f} LOC/SiFP (project weighted average)")
    print(f"  Industry baseline: {industry_loc_per_sifp:.1f} LOC/SiFP")
    print(f"  Project vs Industry: {(project_loc_per_sifp - industry_loc_per_sifp)/industry_loc_per_sifp*100:+.1f}%")
    
    # COMPREHENSIVE STATISTICAL HYPOTHESIS TESTING SUMMARY
    print(f"\n" + "="*70)
    print("COMPREHENSIVE STATISTICAL HYPOTHESIS TESTING")
    print("="*70)
    
    print(f"\nHypothesis: Hallucination-reducing techniques improve time-to-market by >30%")
    print(f"Operational Definition: Multi-stage refinement (actor→judge→meta-judge)")
    print(f"Statistical Significance Threshold: α = 0.05")
    print(f"Practical Significance Threshold: >30% improvement")
    
    # Collect results from all statistical methods
    statistical_evidence = {
        'classical': {},
        'permutation': {},
        'bootstrap': {},
        'bayesian': {}
    }
    
    # 1. Classical Statistics
    print(f"\n" + "-"*50)
    print("1. CLASSICAL STATISTICAL ANALYSIS")
    print("-"*50)
    
    if 'statistical_results' in globals() and statistical_results:
        if 't_test' in statistical_results:
            t_result = statistical_results['t_test']
            print(f"  Method: Independent samples t-test")
            print(f"  t-statistic: {t_result['statistic']:.3f}")
            print(f"  p-value: {t_result['p_value']:.6f}")
            print(f"  95% Confidence Interval: ({t_result['ci_95'][0]:.2f}, {t_result['ci_95'][1]:.2f})")
            print(f"  Statistically significant: {'YES' if t_result['significant'] else 'NO'}")
            
            statistical_evidence['classical'] = {
                'significant': t_result['significant'],
                'p_value': t_result['p_value'],
                'method': 't-test'
            }
        
        if 'effect_size' in statistical_results:
            effect = statistical_results['effect_size']
            print(f"  Effect size (Cohen's d): {effect['cohens_d']:.3f} ({effect['interpretation']} effect)")
            statistical_evidence['classical']['effect_size'] = effect['cohens_d']
    else:
        print(f"  Classical statistical testing: NOT COMPLETED")
        print(f"  Reason: Insufficient data or analysis not run")
    
    # 2. Permutation Tests
    print(f"\n" + "-"*50)
    print("2. PERMUTATION TEST ANALYSIS")
    print("-"*50)
    
    if 'permutation_results' in globals() and permutation_results:
        perm_tests = ['error_reduction', 'success_improvement', 'combined_improvement']
        significant_perm_tests = 0
        total_perm_tests = 0
        
        for test_name in perm_tests:
            if test_name in permutation_results and permutation_results[test_name]:
                total_perm_tests += 1
                result = permutation_results[test_name]
                test_label = test_name.replace('_', ' ').title()
                
                print(f"  {test_label}:")
                print(f"    p-value: {result.get('p_value', 'N/A'):.6f}")
                print(f"    Improvement: {result.get('improvement_pct', 'N/A'):.1f}%")
                print(f"    Significant: {'YES' if result.get('significant', False) else 'NO'}")
                print(f"    Meets >30% threshold: {'YES' if result.get('meets_threshold', False) else 'NO'}")
                
                if result.get('significant', False):
                    significant_perm_tests += 1
        
        print(f"\n  Overall Permutation Results: {significant_perm_tests}/{total_perm_tests} tests significant")
        print(f"  Method advantages: Distribution-free, exact p-values, robust to outliers")
        
        statistical_evidence['permutation'] = {
            'significant_tests': significant_perm_tests,
            'total_tests': total_perm_tests,
            'any_significant': significant_perm_tests > 0
        }
    else:
        print(f"  Permutation testing: NOT AVAILABLE")
        print(f"  Reason: Advanced statistical libraries not installed or analysis not run")
    
    # 3. Bootstrap Analysis
    print(f"\n" + "-"*50)
    print("3. BOOTSTRAP HYPOTHESIS TESTING")
    print("-"*50)
    
    if 'bootstrap_results' in globals() and bootstrap_results:
        boot_tests = ['error_reduction', 'success_improvement']
        threshold_boot_tests = 0
        total_boot_tests = 0
        
        for test_name in boot_tests:
            if test_name in bootstrap_results and bootstrap_results[test_name]:
                total_boot_tests += 1
                result = bootstrap_results[test_name]
                test_label = test_name.replace('_', ' ').title()
                
                print(f"  {test_label}:")
                print(f"    Observed improvement: {result.get('observed_improvement', 'N/A'):.1f}%")
                print(f"    95% Bootstrap CI: [{result.get('confidence_interval', [0,0])[0]:.1f}, {result.get('confidence_interval', [0,0])[1]:.1f}]%")
                print(f"    P(improvement > 30%): {result.get('threshold_confidence', 'N/A'):.3f}")
                print(f"    Meets >30% threshold: {'YES' if result.get('meets_threshold', False) else 'NO'}")
                
                if result.get('meets_threshold', False):
                    threshold_boot_tests += 1
        
        print(f"\n  Overall Bootstrap Results: {threshold_boot_tests}/{total_boot_tests} tests meet >30% threshold")
        print(f"  Method advantages: Direct threshold testing, bias-corrected intervals")
        
        statistical_evidence['bootstrap'] = {
            'threshold_tests': threshold_boot_tests,
            'total_tests': total_boot_tests,
            'any_threshold': threshold_boot_tests > 0
        }
    else:
        print(f"  Bootstrap testing: NOT AVAILABLE")
        print(f"  Reason: Advanced statistical libraries not installed or analysis not run")
    
    # 4. Bayesian Analysis
    print(f"\n" + "-"*50)
    print("4. BAYESIAN HYPOTHESIS TESTING")
    print("-"*50)
    
    if 'bayesian_results' in globals() and bayesian_results:
        print(f"  Method: Bayesian inference with posterior probability statements")
        
        if 'error_reduction' in bayesian_results and bayesian_results['error_reduction']:
            error_result = bayesian_results['error_reduction']
            print(f"  Error Reduction Analysis:")
            print(f"    Posterior mean improvement: {error_result.get('posterior_mean', 'N/A'):.1f}%")
            print(f"    95% Credible interval: [{error_result.get('credible_interval', [0,0])[0]:.1f}, {error_result.get('credible_interval', [0,0])[1]:.1f}]%")
            print(f"    P(improvement > 30%): {error_result.get('prob_30_plus', 'N/A'):.3f}")
            print(f"    P(any improvement): {error_result.get('prob_any_improvement', 'N/A'):.3f}")
        
        if 'success_improvement' in bayesian_results and bayesian_results['success_improvement']:
            success_result = bayesian_results['success_improvement']
            print(f"  Success Rate Analysis:")
            print(f"    P(improvement > 30%): {success_result.get('prob_30_plus', 'N/A'):.3f}")
            print(f"    P(any improvement): {success_result.get('prob_any_improvement', 'N/A'):.3f}")
        
        if 'decision_analysis' in bayesian_results and bayesian_results['decision_analysis']:
            decision_result = bayesian_results['decision_analysis']
            print(f"  Combined Decision Analysis:")
            print(f"    P(either metric > 30%): {decision_result.get('prob_either_30', 'N/A'):.3f}")
            print(f"    P(both metrics > 30%): {decision_result.get('prob_both_30', 'N/A'):.3f}")
            print(f"    Recommendation: {decision_result.get('recommendation', 'N/A')}")
        
        if 'model_comparison' in bayesian_results and bayesian_results['model_comparison']:
            model_result = bayesian_results['model_comparison']
            print(f"  Model Comparison:")
            print(f"    Evidence: {model_result.get('preference', 'N/A')}")
            print(f"    ΔWAIC: {model_result.get('delta_waic', 'N/A'):.2f}")
        
        print(f"  Method advantages: Direct probability statements, credible intervals, decision framework")
        
        # Determine Bayesian evidence strength
        bayes_strong = False
        if ('error_reduction' in bayesian_results and 
            bayesian_results['error_reduction'].get('prob_30_plus', 0) > 0.8):
            bayes_strong = True
        if ('decision_analysis' in bayesian_results and 
            bayesian_results['decision_analysis'].get('prob_either_30', 0) > 0.8):
            bayes_strong = True
            
        statistical_evidence['bayesian'] = {
            'strong_evidence': bayes_strong,
            'available': True
        }
    else:
        print(f"  Bayesian analysis: NOT AVAILABLE")
        print(f"  Reason: PyMC/ArviZ not installed or analysis not run")
        statistical_evidence['bayesian'] = {'available': False}
    
    # INTEGRATED STATISTICAL CONCLUSION
    print(f"\n" + "="*70)
    print("INTEGRATED STATISTICAL CONCLUSION")
    print("="*70)
    
    # Count evidence across methods
    evidence_count = 0
    total_methods = 0
    evidence_details = []
    
    # Classical evidence
    if statistical_evidence['classical']:
        total_methods += 1
        if statistical_evidence['classical'].get('significant', False):
            evidence_count += 1
            evidence_details.append("Classical t-test: SIGNIFICANT")
        else:
            evidence_details.append("Classical t-test: Not significant")
    
    # Permutation evidence
    if statistical_evidence['permutation']:
        total_methods += 1
        if statistical_evidence['permutation'].get('any_significant', False):
            evidence_count += 1
            evidence_details.append("Permutation tests: SIGNIFICANT")
        else:
            evidence_details.append("Permutation tests: Not significant")
    
    # Bootstrap evidence
    if statistical_evidence['bootstrap']:
        total_methods += 1
        if statistical_evidence['bootstrap'].get('any_threshold', False):
            evidence_count += 1
            evidence_details.append("Bootstrap tests: MEETS THRESHOLD")
        else:
            evidence_details.append("Bootstrap tests: Below threshold")
    
    # Bayesian evidence
    if statistical_evidence['bayesian']['available']:
        total_methods += 1
        if statistical_evidence['bayesian'].get('strong_evidence', False):
            evidence_count += 1
            evidence_details.append("Bayesian analysis: STRONG EVIDENCE")
        else:
            evidence_details.append("Bayesian analysis: Weak evidence")
    
    print(f"\nEvidence Summary:")
    for detail in evidence_details:
        print(f"  • {detail}")
    
    print(f"\nStatistical Methods Agreement: {evidence_count}/{total_methods} methods show positive evidence")
    
    # Overall statistical conclusion
    if evidence_count >= total_methods * 0.75:
        statistical_conclusion = "STRONG STATISTICAL EVIDENCE"
        conclusion_color = "🟢"
    elif evidence_count >= total_methods * 0.5:
        statistical_conclusion = "MODERATE STATISTICAL EVIDENCE"
        conclusion_color = "🟡"
    else:
        statistical_conclusion = "WEAK STATISTICAL EVIDENCE"
        conclusion_color = "🔴"
    
    print(f"\n{conclusion_color} OVERALL STATISTICAL CONCLUSION: {statistical_conclusion}")
    print(f"   for >30% improvement in time-to-market through hallucination-reducing techniques")
    
    # MODEL PERFORMANCE AND BUSINESS IMPACT
    print(f"\n" + "="*70)
    print("MODEL PERFORMANCE AND BUSINESS RECOMMENDATIONS")
    print("="*70)
    
    if not performance_df.empty:
        # Best performing models
        best_accuracy = performance_df.loc[performance_df['Error_Pct'].idxmin()]
        best_coverage = performance_df.loc[performance_df['Success_Rate'].idxmax()]
        
        print(f"\n1. RECOMMENDED MODELS:")
        print(f"   Most Accurate Model: {best_accuracy['Model']}")
        print(f"     • Error rate: {best_accuracy['Error_Pct']:.1f}%")
        print(f"     • Success rate: {best_accuracy['Success_Rate']:.1%}")
        print(f"     • LOC per SiFP: {best_accuracy['LOC_per_SiFP']:.1f}")
        
        print(f"\n   Best Coverage Model: {best_coverage['Model']}")
        print(f"     • Success rate: {best_coverage['Success_Rate']:.1%}")
        print(f"     • Error rate: {best_coverage['Error_Pct']:.1f}%")
        print(f"     • LOC per SiFP: {best_coverage['LOC_per_SiFP']:.1f}")
        
        if 'effort_impact_df' in globals() and not effort_impact_df.empty:
            best_effort = effort_impact_df.loc[effort_impact_df['Effort_Error_Pct'].abs().idxmin()]
            print(f"\n   Best Effort Estimation Model: {best_effort['Model']}")
            print(f"     • Effort error: {best_effort['Effort_Error_Pct']:+.1f}%")
            print(f"     • Cost impact: ${best_effort['Total_Cost_Impact_USD']:+,.0f}")
    
    # Business impact summary
    if 'effort_impact_df' in globals() and not effort_impact_df.empty:
        print(f"\n2. BUSINESS IMPACT ANALYSIS:")
        total_cost_impact = effort_impact_df['Total_Cost_Impact_USD'].sum()
        avg_effort_error = effort_impact_df['Effort_Error_Pct'].mean()
        max_cost_impact = effort_impact_df['Total_Cost_Impact_USD'].abs().max()
        
        print(f"   Total net cost impact: ${total_cost_impact:+,.0f}")
        print(f"   Average effort estimation error: {avg_effort_error:+.1f}%")
        print(f"   Maximum cost impact (single model): ${max_cost_impact:,.0f}")
        print(f"   Cost per hour assumed: ${CONFIG.get('COST_PER_HOUR', 100)}/hour")
        
        # Risk assessment
        print(f"\n3. RISK ASSESSMENT:")
        if abs(avg_effort_error) < 15:
            print(f"   ✅ LOW RISK: Average estimation errors within acceptable range (<15%)")
        else:
            print(f"   ⚠️  MODERATE RISK: Average estimation errors exceed 15% threshold")
        
        if abs(total_cost_impact) < 50000:
            print(f"   ✅ LOW FINANCIAL RISK: Total cost impact manageable (<$50k)")
        else:
            print(f"   ⚠️  FINANCIAL RISK: Significant cost impact requires attention")
    
    # IMPLEMENTATION RECOMMENDATIONS
    print(f"\n" + "="*70)
    print("IMPLEMENTATION RECOMMENDATIONS")
    print("="*70)
    
    print(f"\n1. STATISTICAL EVIDENCE BASED:")
    if evidence_count >= total_methods * 0.75:
        print(f"   🚀 STRONG RECOMMENDATION: Implement hallucination-reducing techniques")
        print(f"      • Multiple statistical methods converge on positive evidence")
        print(f"      • Risk of Type I error minimized through diverse analytical approaches")
        print(f"      • Expected improvement supported by robust statistical framework")
    elif evidence_count >= total_methods * 0.5:
        print(f"   📋 MODERATE RECOMMENDATION: Pilot implementation with careful monitoring")
        print(f"      • Mixed statistical evidence suggests cautious adoption")
        print(f"      • Implement in controlled environment with measurement systems")
        print(f"      • Establish clear success metrics and decision criteria")
    else:
        print(f"   ⚠️  WEAK RECOMMENDATION: Additional research needed before implementation")
        print(f"      • Insufficient statistical evidence for confident recommendation")
        print(f"      • Consider larger sample sizes or alternative approaches")
        print(f"      • Focus on improving measurement and data collection systems")
    
    print(f"\n2. TECHNICAL IMPLEMENTATION:")
    print(f"   • Multi-stage refinement: actor → judge → meta-judge architecture")
    print(f"   • Quality scoring systems for estimation validation")
    print(f"   • Continuous monitoring of estimation accuracy")
    print(f"   • Regular model performance evaluation and recalibration")
    
    print(f"\n3. ORGANIZATIONAL READINESS:")
    print(f"   • Training on new estimation methodologies")
    print(f"   • Integration with existing project management processes")
    print(f"   • Change management for adoption of AI-assisted estimation")
    print(f"   • Establishment of governance and quality assurance protocols")
    
    # LIMITATIONS AND FUTURE RESEARCH
    print(f"\n" + "="*70)
    print("LIMITATIONS AND FUTURE RESEARCH DIRECTIONS")
    print("="*70)
    
    print(f"\nStudy Limitations:")
    print(f"  • Sample size constraints limit statistical power")
    print(f"  • Cross-sectional design rather than longitudinal study")
    print(f"  • Proxy measures for 'hallucination-reducing techniques'")
    print(f"  • Project-specific results may not generalize broadly")
    print(f"  • Limited direct measurement of time-to-market impact")
    
    print(f"\nFuture Research Priorities:")
    print(f"  1. Randomized controlled trials with larger sample sizes")
    print(f"  2. Direct measurement of time-to-market metrics in live projects")
    print(f"  3. Multi-organization validation across different domains")
    print(f"  4. Longitudinal studies tracking improvement sustainability")
    print(f"  5. Cost-benefit analysis of implementation overhead")
    print(f"  6. Comparative analysis with alternative estimation methods")
    
    # FINAL EXECUTIVE DECISION FRAMEWORK
    print(f"\n" + "="*70)
    print("EXECUTIVE DECISION FRAMEWORK")
    print("="*70)
    
    print(f"\n🎯 KEY FINDINGS:")
    print(f"   • Statistical Analysis: {statistical_conclusion}")
    print(f"   • Methods Agreement: {evidence_count}/{total_methods} approaches show positive results")
    if not performance_df.empty:
        print(f"   • Best Model Accuracy: {performance_df['Error_Pct'].min():.1f}% error rate")
        print(f"   • Model Consistency: {'High' if performance_df['Error_Pct'].std() < 10 else 'Moderate'}")
    if 'effort_impact_df' in globals() and not effort_impact_df.empty:
        print(f"   • Financial Impact: ${effort_impact_df['Total_Cost_Impact_USD'].sum():+,.0f} net impact")
    
    print(f"\n📊 DECISION MATRIX:")
    print(f"   Statistical Evidence: {'Strong' if evidence_count >= total_methods * 0.75 else 'Moderate' if evidence_count >= total_methods * 0.5 else 'Weak'}")
    print(f"   Implementation Risk: {'Low' if abs(avg_effort_error) < 15 else 'Moderate'}")
    print(f"   Financial Impact: {'Manageable' if abs(total_cost_impact) < 50000 else 'Significant'}")
    print(f"   Technical Readiness: {'High' if not performance_df.empty else 'Moderate'}")
    
    print(f"\n🎯 EXECUTIVE RECOMMENDATION:")
    if evidence_count >= total_methods * 0.75:
        print(f"   PROCEED WITH IMPLEMENTATION")
        print(f"   • Strong statistical evidence supports adoption")
        print(f"   • Risk-reward profile favorable for implementation")
        print(f"   • Establish monitoring systems for continuous validation")
    elif evidence_count >= total_methods * 0.5:
        print(f"   PROCEED WITH PILOT PROGRAM")
        print(f"   • Moderate evidence supports cautious adoption")
        print(f"   • Implement with robust measurement and evaluation")
        print(f"   • Establish clear go/no-go criteria for full deployment")
    else:
        print(f"   DEFER IMPLEMENTATION - CONDUCT ADDITIONAL RESEARCH")
        print(f"   • Insufficient evidence for confident business decision")
        print(f"   • Invest in larger-scale validation studies")
        print(f"   • Focus on improving data collection and measurement systems")
    
    # Save comprehensive results
    try:
        os.makedirs('results', exist_ok=True)
        timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
        
        # Create comprehensive executive summary
        final_summary = {
            'project': project_name,
            'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
            'hypothesis': 'Hallucination-reducing techniques improve time-to-market by >30%',
            'statistical_methods_used': total_methods,
            'methods_with_positive_evidence': evidence_count,
            'overall_statistical_conclusion': statistical_conclusion,
            'evidence_strength_ratio': f"{evidence_count}/{total_methods}",
            'best_accuracy_model': performance_df.loc[performance_df['Error_Pct'].idxmin(), 'Model'] if not performance_df.empty else 'N/A',
            'best_accuracy_error': float(performance_df['Error_Pct'].min()) if not performance_df.empty else 'N/A',
            'total_cost_impact': float(effort_impact_df['Total_Cost_Impact_USD'].sum()) if 'effort_impact_df' in globals() and not effort_impact_df.empty else 'N/A',
            'executive_recommendation': 'PROCEED WITH IMPLEMENTATION' if evidence_count >= total_methods * 0.75 else 'PROCEED WITH PILOT PROGRAM' if evidence_count >= total_methods * 0.5 else 'DEFER IMPLEMENTATION',
            'statistical_evidence_details': evidence_details,
            'analysis_timestamp': timestamp
        }
        
        # Save detailed results
        import json
        with open(f'results/enhanced_executive_summary_{project_name}_{timestamp}.json', 'w') as f:
            json.dump(final_summary, f, indent=2, default=str)
        
        # Save statistical evidence summary
        with open(f'results/statistical_evidence_{project_name}_{timestamp}.json', 'w') as f:
            json.dump(statistical_evidence, f, indent=2, default=str)
        
        print(f"\n✅ COMPREHENSIVE ANALYSIS RESULTS SAVED")
        print(f"   Location: results/enhanced_executive_summary_{project_name}_{timestamp}.json")
        print(f"   Statistical Evidence: results/statistical_evidence_{project_name}_{timestamp}.json")
        
    except Exception as e:
        print(f"\n⚠️  Warning: Could not save results - {e}")

else:
    print("❌ ANALYSIS INCOMPLETE")
    print("Missing required data components:")
    print("  - performance_df: Model performance metrics")
    print("  - effort_impact_df: Effort and cost impact analysis")
    print("  - statistical_results: Basic hypothesis testing results")
    print("\nPlease ensure all previous analysis cells have completed successfully.")

print(f"\n" + "="*70)
print("🎉 ENHANCED STATISTICAL ANALYSIS COMPLETE")
print("="*70)
print("Framework: Classical → Permutation → Bootstrap → Bayesian → Integration")
print("Thank you for using the Enhanced SiFP COSMIC Estimation Analysis Framework!")
print("For questions or support, refer to the comprehensive documentation and statistical methodologies.")